## Imports

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import pypsa
import math
import seaborn as sns
import cartopy
import cartopy.crs as ccrs
import matplotlib
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from shapely.geometry import Point, LineString
from datetime import datetime
import matplotlib.patheffects as pe
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
from matplotlib.lines import Line2D
import matplotlib.ticker as mtick

from datetime import date, datetime, time, timedelta

# imported own functions
from utils import market_values, market_values_by_time_index, nodal_balance, capacity, capacity_links, capacity_storage_units, get_condense_sum, nodal_balance, generation, generation_links, generation_storage_units, market_values_storage_units, market_values_links, time_stored_LIFO

# imported own definitions
from utils import carrier_colors, carrier_renaming, carrier_renaming_reverse 
from utils import resistive_heater, gas_boiler, heat_pump, water_tanks_charger, water_tanks_discharger, solar_thermal
from utils import c_el_gen_s, c_el_con_s

# general variables
font1 = {'fontname':'Calibri'}
PLOT_DIR = 'C:/Users/Julian/Studies/Master/01 TU Berlin/3. Semester - Masterarbeit/MA Marktwerte FEE/data/plots/'
onshore_regions = gpd.read_file("../data/external/regions_onshore_elec_s_181.geojson")
offshore_regions = gpd.read_file("../data/external/regions_offshore_elec_s_181.geojson")
onshore_regions = onshore_regions.set_index('name')
offshore_regions = offshore_regions.set_index('name')

In [ ]:
# Network imports
n_no =pypsa.Network("../data/raw/elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2030.nc")
n_h2 =pypsa.Network("../data/raw/elec_s_181_lvopt__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10_2030.nc")

In [ ]:
# stst and exp dataframes
# spatial
df_stst_ons = pd.read_pickle("../data/processed/df_stst_ons.pkl")
df_stst_off = pd.read_pickle("../data/processed/df_stst_off.pkl")
df_exp_ons = pd.read_pickle("../data/processed/df_exp_ons.pkl")
df_exp_off = pd.read_pickle("../data/processed/df_exp_off.pkl")

# temporal
df_stst_ts = pd.read_pickle("../data/processed/df_stst_ts.pkl")
df_exp_ts = pd.read_pickle("../data/processed/df_exp_ts.pkl")

# other
overall_ur_stst = pd.read_pickle("../data/processed/overall_ur_stst.pkl")
overall_ur_exp = pd.read_pickle("../data/processed/overall_ur_exp.pkl")

In [ ]:
# Notebook Definitions
c1_groups = [resistive_heater, gas_boiler, heat_pump, water_tanks_charger, water_tanks_discharger, solar_thermal]
c1_groups_name = ["resistive heater", "gas boiler", "heat pump", "water tanks charger", "water tanks discharger", "solar thermal"]
markers =["v", "^", "<", ">", "1", "2", "3", "4", "*", "+", "d", "o", "|", "s", "P", "p", "h"]

In [ ]:
# Notebook Functions

def get_df(df_no, df_h2, carriers):
    result = pd.DataFrame(index = ["STST", "EXP"])

    for c in carriers:
        result.loc["STST" , c] = df_no[c].values
        result.loc["EXP" , c] = df_h2[c].values
    return result

def c_gen(carrier_list, network):
    result = []
    for c in carrier_list:
        if c in network.generators.carrier.unique().tolist():
            result.append(c)
    return result

def c_link(carrier_list, network):
    result = []
    for c in carrier_list:
        if c in network.links.carrier.unique().tolist():
            result.append(c)
    return result

def c_su(carrier_list, network):
    result = []
    for c in carrier_list:
        if c in network.storage_units.carrier.unique().tolist():
            result.append(c)
    return result

In [ ]:
# Regions

onshore_regions['coords'] = onshore_regions['geometry'].apply(lambda x: x.representative_point().coords[:])
onshore_regions['coords'] = [coords[0] for coords in onshore_regions['coords']]
onshore_regions["name"] = onshore_regions.index
offshore_regions['coords'] = offshore_regions['geometry'].apply(lambda x: x.representative_point().coords[:])
offshore_regions['coords'] = [coords[0] for coords in offshore_regions['coords']]
offshore_regions["name"] = offshore_regions.index

## CALC

In [ ]:
# calc market values, generation, lmps, capacity factors for generators, links and storage units

for n in [n_no, n_h2]:
    df_regions_onshore = onshore_regions.copy()
    df_regions_offshore = offshore_regions.copy()

    # function for carriers in n.generators.carrier.unique() #13
    for carrier in n.generators.carrier.unique():
        df_regions_onshore[f"{carrier}_mv"] = market_values(n, carrier)
        df_regions_offshore[f"{carrier}_mv"] = market_values(n, carrier)
        # generation in TWh (does this have to be multiplied by 3??
        df_regions_onshore[f"{carrier}_gen"] = generation(n, carrier) / 1000 * 3
        df_regions_offshore[f"{carrier}_gen"] = generation(n, carrier) / 1000 * 3
        # lmps
        # capacity factors This calculation is correct? as capacity is multiplied by 2920 is the same as multiplying the generation by 3 and then dividing it by the capacity times 8760 (as cap is in MWh?)
        df_regions_onshore[f"{carrier}_cf"] = generation(n, carrier) / (capacity(n, carrier) *2920)
        df_regions_offshore[f"{carrier}_cf"] = generation(n, carrier) / (capacity(n, carrier) *2920)


    # function for carriers in n.links.carrier.unique() #53
    for carrier in n.links.carrier.unique():
        df_regions_onshore[f"{carrier}_mv"] = market_values_links(n, carrier)
        df_regions_onshore[f"{carrier}_gen"] = generation_links(n, carrier) / 1000 * 3
        df_regions_onshore[f"{carrier}_cf"] = generation_links(n, carrier) / (capacity_links(n, carrier) *2920)

    # function for carriers in n.storage_units.carrier.unique() #2
    for carrier in n.storage_units.carrier.unique():
        df_regions_onshore[f"{carrier}_mv"] = market_values_storage_units(n, carrier)
        df_regions_onshore[f"{carrier}_gen"] = generation_storage_units(n, carrier) / 1000 * 3
        # capacity factors (both generation and consumption(loading) is considered
        gen = abs(n.storage_units_t.p.loc[:, n.storage_units.carrier == carrier])
        gen.columns = gen.columns.map(n.storage_units.bus)
        gen.columns = gen.columns.map(n.buses.location)
        df_regions_onshore[f"{carrier}_cf"] = gen.sum() / (capacity_storage_units(n, carrier) *2920)

    # set market values to nan where generation in corresponding region is lower than % quantile
    qt = 0.2
    for carrier in (n.generators.carrier.unique().tolist() +
                    n.links.carrier.unique().tolist() +
                    n.storage_units.carrier.unique().tolist()):
        index = df_regions_onshore[f"{carrier}_gen"] <= np.quantile(df_regions_onshore[f"{carrier}_gen"], qt)
        df_regions_onshore[f"{carrier}_mv"][index] = np.nan

    # calc lmps at the buses (lmps that are only present for EU (e.g. oil) are nan at the moment)
    # TODO: decide if EU lmps are used as lmp for all regions
    for carrier_bus in n.buses.carrier.unique():
        # index would be same names as the bus (not the location as it is in the index of
        # df_regions_onshore so far -> map location to make sure the right lmp is set
        locs = n.buses.location[n.buses[n.buses.carrier == carrier_bus].index]
        lmps = n.buses_t.marginal_price[n.buses[n.buses.carrier == carrier_bus].index].mean()
        df = pd.concat([lmps, locs], axis=1).rename(columns={0: f"{carrier_bus}_lmp"})
        df.set_index("location", inplace=True)
        if df.size == 1:
            if df.index == "EU":
                df = pd.DataFrame(np.repeat(df.values, 181), index=df_regions_onshore.index, columns=[f"{carrier_bus}_lmp"])
                df_regions_onshore[f"{carrier_bus}_lmp"] = df[f"{carrier_bus}_lmp"]
        else:
            df_regions_onshore[f"{carrier_bus}_lmp"] = df[f"{carrier_bus}_lmp"]

    if n == n_no:
        df_no_ons = df_regions_onshore
        df_no_off = df_regions_offshore

    if n == n_h2:
        df_h2_ons = df_regions_onshore
        df_h2_off = df_regions_offshore

df_no_ons.head()

In [ ]:
#assert 0


# 4.2 Important Technologies and their role

## Electricity Balances

In [ ]:
carrier = ["AC", "low voltage"]
loads = ["electricity", "industry electricity", "agriculture electricity"]
period = "2013-05"
nb_el_no = nodal_balance(n_no, carrier = carrier, time=period, aggregate=['component', 'bus'], energy=True)  # in units of energy
nb_el_h2 = nodal_balance(n_h2, carrier = carrier, time=period, aggregate=['component', 'bus'], energy=True)  # in units of energy
# convert from MW to GW and unstack
nb_el_no = nb_el_no.unstack(level=[1]) / 1000
nb_el_h2 = nb_el_h2.unstack(level=[1]) / 1000
loads_el_no = abs(nb_el_no[loads].sum(axis=1))
loads_el_h2 = abs(nb_el_h2[loads].sum(axis=1))
#nb_el_no.drop(loads, axis=1, inplace=True)
#nb_el_h2.drop(loads, axis=1, inplace=True)

# condense groups
nb_el_no = get_condense_sum(nb_el_no, c1_groups, c1_groups_name)
nb_el_h2 = get_condense_sum(nb_el_h2, c1_groups, c1_groups_name)
# rename unhandy column names
nb_el_no.rename(columns=carrier_renaming, inplace=True)
nb_el_h2.rename(columns=carrier_renaming, inplace=True)

nb_el_no.head(3)

In [ ]:
# differences
nb_el_no.columns.difference(nb_el_h2.columns)

- network without H2 grid has no 'biomass CHP'
- network with H2 grid has no Direct Air Capture, no H2 Fuel Cell, no gas CHP and no gas CHP CC

In [ ]:
# thesis_plot

fig, ax = plt.subplots(figsize=(14, 8))

model = "EXP"

if model == "STST":
    n = n_no
    df = nb_el_no
    df_loads = loads_el_no
elif model == "EXP":
    n = n_h2
    df = nb_el_h2
    df_loads = loads_el_h2

# split into df with positive and negative values
df_neg, df_pos = df.clip(upper=0), df.clip(lower=0)
# exclude all technologies that contribute less than th
th = 0.005
df_pos_share = df_pos.sum() / df_pos.sum().sum()
df_pos = df_pos[df_pos_share[df_pos_share > th].sort_values(ascending=False).index]
df_neg_share = df_neg.sum() / df_neg.sum().sum()
df_neg = df_neg[df_neg_share[df_neg_share > th].sort_values(ascending=False).index]
# get colors
c_neg, c_pos = [carrier_colors[col] for col in df_neg.columns], [carrier_colors[col] for col in df_pos.columns]

# plot positive values
ax = df_pos.plot.area(ax=ax, stacked=True, color=c_pos, linewidth=0.)

# rename negative values that are also present on positive side, so that they are not shown and plot negative values
f = lambda c: '_' + c
cols = [f(c) if (c in df_pos.columns) else c for c in  df_neg.columns]
cols_map = dict(zip(df_neg.columns, cols))
ax = df_neg.rename(columns=cols_map).plot.area(ax=ax, stacked=True, color=c_neg, linewidth=0.)

# plot lmps
lmps = n.buses_t.marginal_price[n.buses[n.buses.carrier.isin(carrier)].index].mean(axis=1)[period]
ax2 = lmps.plot(style="--", color="black", label="lmp (mean over buses)", secondary_y=True)
ax2.grid(False)
# set limits of secondary y-axis
ax2.set_ylim([ - 1.5 * lmps.max() * abs(df_neg.sum(axis=1).min()) /  df_pos.sum(axis=1).max() , 1.5 * lmps.max()])

# plot loads
df_loads.plot(style=":", color="black", label="electricity loads")

# rescale the y-axis
ax.set_ylim([1.05*df_neg.sum(axis=1).min(), 1.05*df_pos.sum(axis=1).max()])
ax.legend(ncol=1, loc="upper center", bbox_to_anchor=(1.17, 1.01), title="Legend for left y-axis")
ax2.legend(title="Legend for right y-axis",  loc="upper right")
ax.set_ylabel("total electriyity balance [GW]")
ax2.set_ylabel("lmp [€/MWh]")
ax.set_xlabel("")
ax.set_title(f"Electricity balance May ({model})", fontsize=16, pad=15,  **font1)
ax.grid(True)
fig.tight_layout()

#plt.close()
plt.show()

fig.savefig(f"{PLOT_DIR}01_general/4.2_systems_technologies/el_bal_may_{model}.png")

#Further analysis

In [ ]:
# check if technologies are ein plot which are not in index
set(df_pos.columns).difference(c_el_gen_s)
set(c_el_gen_s).difference(df_pos.columns)
# gen: 'PHS', 'urban central gas CHP', 'urban central solid biomass CHP CC' are not in plot but in index
# con: 'urban central air heat pump', 'urban central resistive heater' are present aggregated

In [ ]:
set(df_neg.columns).difference(c_el_con_s)

In [ ]:
df_pos.sum()

- onwind only slighlty with most generation (not the case for June, July and August)
onwind,345562.363252
solar,338931.403660

In [ ]:
df_neg

In [ ]:
# correlation of solar generation and lmp
nb_el_no[["solar", "solar rooftop"]].sum(axis=1).corr(lmps) # -0.6751
nb_el_no.onwind.corr(lmps) # -0.358751
nb_el_no.PHS.corr(lmps) # 0.78373
nb_el_no.PHS.corr(nb_el_no.solar) # -0.9022

In [ ]:
# range of lmps
lmps.describe()

In [ ]:
# range of lmps
lmps.describe()

### Electricity Balance (Jan)

In [ ]:
carrier = ["AC", "low voltage"]
loads = ["electricity", "industry electricity", "agriculture electricity"]
period = "2013-01"
nb_el_no = nodal_balance(n_no, carrier = carrier, time=period, aggregate=['component', 'bus'], energy=True)  # in units of energy
nb_el_h2 = nodal_balance(n_h2, carrier = carrier, time=period, aggregate=['component', 'bus'], energy=True)  # in units of energy
# convert from MW to GW and unstack
nb_el_no = nb_el_no.unstack(level=[1]) / 1000
nb_el_h2 = nb_el_h2.unstack(level=[1]) / 1000
loads_el_no = abs(nb_el_no[loads].sum(axis=1))
loads_el_h2 = abs(nb_el_h2[loads].sum(axis=1))
#nb_el_no.drop(loads, axis=1, inplace=True)
#nb_el_h2.drop(loads, axis=1, inplace=True)

# condense groups
nb_el_no = get_condense_sum(nb_el_no, c1_groups, c1_groups_name)
nb_el_h2 = get_condense_sum(nb_el_h2, c1_groups, c1_groups_name)
# rename unhandy column names
nb_el_no.rename(columns=carrier_renaming, inplace=True)
nb_el_h2.rename(columns=carrier_renaming, inplace=True)

nb_el_no.head(3)

In [ ]:
# thesis plot

fig, ax = plt.subplots(figsize=(14, 8))

model = "STST"

if model == "STST":
    n = n_no
    df = nb_el_no
    df_loads = loads_el_no
elif model == "EXP":
    n = n_h2
    df = nb_el_h2
    df_loads = loads_el_h2


# split into df with positive and negative values
df_neg, df_pos = df.clip(upper=0), df.clip(lower=0)
# exclude all technologies that contribute less than th
th = 0.005
df_pos_share = df_pos.sum() / df_pos.sum().sum()
df_pos = df_pos[df_pos_share[df_pos_share > th].sort_values(ascending=False).index]
df_neg_share = df_neg.sum() / df_neg.sum().sum()
df_neg = df_neg[df_neg_share[df_neg_share > th].sort_values(ascending=False).index]
# get colors
c_neg, c_pos = [carrier_colors[col] for col in df_neg.columns], [carrier_colors[col] for col in df_pos.columns]

# plot positive values
ax = df_pos.plot.area(ax=ax, stacked=True, color=c_pos, linewidth=0.)

# rename negative values that are also present on positive side, so that they are not shown and plot negative values
f = lambda c: '_' + c
cols = [f(c) if (c in df_pos.columns) else c for c in  df_neg.columns]
cols_map = dict(zip(df_neg.columns, cols))
ax = df_neg.rename(columns=cols_map).plot.area(ax=ax, stacked=True, color=c_neg, linewidth=0.)

# plot lmps
lmps = n.buses_t.marginal_price[n.buses[n.buses.carrier.isin(carrier)].index].mean(axis=1)[period]
ax2 = lmps.plot(style="--", color="black", label="lmp (mean over buses)", secondary_y=True)
ax2.grid(False)
# set limits of secondary y-axis
ax2.set_ylim([ - 1.5 * lmps.max() * abs(df_neg.sum(axis=1).min()) /  df_pos.sum(axis=1).max() , 1.5 * lmps.max()])

# plot loads
df_loads.plot(style=":", color="black", label="electricity loads")

# rescale the y-axis
ax.set_ylim([1.05*df_neg.sum(axis=1).min(), 1.05*df_pos.sum(axis=1).max()])
ax.legend(ncol=1, loc="upper center", bbox_to_anchor=(1.17, 1.01), title="Legend for left y-axis")
ax2.legend(title="Legend for right y-axis",  loc="upper right")
ax.set_ylabel("total electriyity balance [GW]")
ax2.set_ylabel("lmp [€/MWh]")
ax.set_xlabel("")
ax.grid(True)
ax.set_title(f"Electricity balance January ({model})", fontsize=16, pad=15,  **font1)
fig.tight_layout()

#plt.close()
plt.show()

fig.savefig(f"{PLOT_DIR}01_general/4.2_systems_technologies/el_bal_jan_{model}.png")

In [ ]:
# range of lmps
lmps.describe()

In [ ]:
# range of lmps
lmps.describe()

### Electricity balance (all year)

In [ ]:
carrier = ["AC", "low voltage"]
loads = ["electricity", "industry electricity", "agriculture electricity"]
period = "2013"
nb_el_no = nodal_balance(n_no, carrier = carrier, time=period, aggregate=['component', 'bus'], energy=True)  # in units of energy
nb_el_h2 = nodal_balance(n_h2, carrier = carrier, time=period, aggregate=['component', 'bus'], energy=True)  # in units of energy
# convert from MW to GW and unstack
nb_el_no = nb_el_no.unstack(level=[1]) / 1000
nb_el_h2 = nb_el_h2.unstack(level=[1]) / 1000
loads_el_no = abs(nb_el_no[loads].sum(axis=1))
loads_el_h2 = abs(nb_el_h2[loads].sum(axis=1))
nb_el_no.drop(loads, axis=1, inplace=True)
nb_el_h2.drop(loads, axis=1, inplace=True)

# condense groups
nb_el_no_all = get_condense_sum(nb_el_no, c1_groups, c1_groups_name)
nb_el_h2_all = get_condense_sum(nb_el_h2, c1_groups, c1_groups_name)
# rename unhandy column names
nb_el_no_all.rename(columns=carrier_renaming, inplace=True)
nb_el_h2_all.rename(columns=carrier_renaming, inplace=True)

nb_el_no_all.head(3)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))

n = n_no
# "8h", "D", "W", "M"
res = "D"
df = nb_el_no_all#.resample(res).sum()
df_loads = loads_el_no#.resample(res).sum()

# split into df with positive and negative values and get colors
df_neg, df_pos = df.clip(upper=0), df.clip(lower=0)
c_neg, c_pos = [carrier_colors[col] for col in df_neg.columns], [carrier_colors[col] for col in df_pos.columns]

# plot positive values
ax = df_pos.plot.area(ax=ax, stacked=True, color=c_pos, linewidth=0.)

# rename negative values so that they are not shown and plot negative values
ax = df_neg.rename(columns=lambda x: '_' + x).plot.area(ax=ax, stacked=True, color=c_neg, linewidth=0.)

# plot lmps
lmps = n.buses_t.marginal_price[n.buses[n.buses.carrier.isin(carrier)].index].mean(axis=1)[period].resample(res).mean()
ax2 = lmps.plot(style="--", color="black", label="lmp (mean over buses) [€/MWh]", secondary_y=True)
ax2.grid(False)

# plot loads
df_loads.plot(style=":", color="black", label="electricity loads [GWh]")

# set limits of secondary y-axis
ax2.set_ylim([ - 1.5 * lmps.max() * abs(df_neg.sum(axis=1).min()) /  df_pos.sum(axis=1).max() , 1.5 * lmps.max()])

# rescale the y-axis
ax.set_ylim([1.05*df_neg.sum(axis=1).min(), 1.05*df_pos.sum(axis=1).max()])
ax.legend(ncol=1, loc="upper center", bbox_to_anchor=(1.15, 1), title="Legend for left y-axis")
ax2.legend(title="Legend for right y-axis",  loc="upper right")
ax.set_ylabel("tota electriyity balance [GWh]")
ax.set_xlabel("")
ax.grid(True)
ax.set_title(f"{carrier}")
fig.tight_layout()

#plt.close()
plt.show()

In [ ]:
# electricity balance all year with different resolutions

fig, ax = plt.subplots(figsize=(14, 8))

n = n_h2
# "8h", "D", "W", "M"
res = "D"
df = nb_el_h2_all.resample(res).sum()
df_loads = loads_el_h2.resample(res).sum()

# split into df with positive and negative values and get colors
df_neg, df_pos = df.clip(upper=0), df.clip(lower=0)
c_neg, c_pos = [carrier_colors[col] for col in df_neg.columns], [carrier_colors[col] for col in df_pos.columns]

# plot positive values
ax = df_pos.plot.area(ax=ax, stacked=True, color=c_pos, linewidth=0.)

# rename negative values so that they are not shown and plot negative values
ax = df_neg.rename(columns=lambda x: '_' + x).plot.area(ax=ax, stacked=True, color=c_neg, linewidth=0.)

# plot lmps
lmps = n.buses_t.marginal_price[n.buses[n.buses.carrier.isin(carrier)].index].mean(axis=1)[period].resample(res).mean()
ax2 = lmps.plot(style="--", color="black", label="lmp (mean over buses) [€/MWh]", secondary_y=True)
ax2.grid(False)

# plot loads
df_loads.plot(style=":", color="black", label="electricity loads [GWh]")

# set limits of secondary y-axis
ax2.set_ylim([ - 1.5 * lmps.max() * abs(df_neg.sum(axis=1).min()) /  df_pos.sum(axis=1).max() , 1.5 * lmps.max()])

# rescale the y-axis
ax.set_ylim([1.05*df_neg.sum(axis=1).min(), 1.05*df_pos.sum(axis=1).max()])
ax.legend(ncol=1, loc="upper center", bbox_to_anchor=(1.15, 1), title="Legend for left y-axis")
ax2.legend(title="Legend for right y-axis",  loc="upper right")
ax.set_ylabel("tota electriyity balance [GWh]")
ax.set_xlabel("")
ax.grid(True)
ax.set_title(f"{carrier}")
fig.tight_layout()

### Electricity Balances special cases

In [ ]:
carrier = ["AC", "low voltage"]
loads = ["electricity", "industry electricity", "agriculture electricity"]
start_date = "2013-01-22 00:00:00"
end_date = "2013-01-26 00:00:00"
period = n_no.generators_t.p.index[(n_no.generators_t.p.index >= start_date) & (n_no.generators_t.p.index <= end_date)]
nb_el_no = nodal_balance(n_no, carrier = carrier, time=period, aggregate=['component', 'bus'], energy=True)  # in units of energy
nb_el_h2 = nodal_balance(n_h2, carrier = carrier, time=period, aggregate=['component', 'bus'], energy=True)  # in units of energy
# convert from MW to GW and unstack
nb_el_no = nb_el_no.unstack(level=[1]) / 1000
nb_el_h2 = nb_el_h2.unstack(level=[1]) / 1000
loads_el_no = abs(nb_el_no[loads].sum(axis=1))
loads_el_h2 = abs(nb_el_h2[loads].sum(axis=1))
#nb_el_no.drop(loads, axis=1, inplace=True)
#nb_el_h2.drop(loads, axis=1, inplace=True)

# condense groups
nb_el_no = get_condense_sum(nb_el_no, c1_groups, c1_groups_name)
nb_el_h2 = get_condense_sum(nb_el_h2, c1_groups, c1_groups_name)
# rename unhandy column names
nb_el_no.rename(columns=carrier_renaming, inplace=True)
nb_el_h2.rename(columns=carrier_renaming, inplace=True)

nb_el_no.head(3)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))

model = "EXP"

if model == "STST":
    n = n_no
    df = nb_el_no
    df_loads = loads_el_no
elif model == "EXP":
    n = n_h2
    df = nb_el_h2
    df_loads = loads_el_h2

# split into df with positive and negative values
df_neg, df_pos = df.clip(upper=0), df.clip(lower=0)
# exclude all technologies that contribute less than th
th = 0.005
df_pos_share = df_pos.sum() / df_pos.sum().sum()
df_pos = df_pos[df_pos_share[df_pos_share > th].sort_values(ascending=False).index]
df_neg_share = df_neg.sum() / df_neg.sum().sum()
df_neg = df_neg[df_neg_share[df_neg_share > th].sort_values(ascending=False).index]
# get colors
c_neg, c_pos = [carrier_colors[col] for col in df_neg.columns], [carrier_colors[col] for col in df_pos.columns]

# plot positive values
ax = df_pos.plot.area(ax=ax, stacked=True, color=c_pos, linewidth=0.)

# rename negative values that are also present on positive side, so that they are not shown and plot negative values
f = lambda c: '_' + c
cols = [f(c) if (c in df_pos.columns) else c for c in  df_neg.columns]
cols_map = dict(zip(df_neg.columns, cols))
ax = df_neg.rename(columns=cols_map).plot.area(ax=ax, stacked=True, color=c_neg, linewidth=0.)

# plot lmps
lmps = n.buses_t.marginal_price[n.buses[n.buses.carrier.isin(carrier)].index].mean(axis=1)[period]
ax2 = lmps.plot(style="--", color="black", label="lmp (mean over buses)", secondary_y=True)
ax2.grid(False)
# set limits of secondary y-axis
ax2.set_ylim([ - 1.5 * lmps.max() * abs(df_neg.sum(axis=1).min()) /  df_pos.sum(axis=1).max() , 1.5 * lmps.max()])

# plot loads
df_loads.plot(style=":", color="black", label="electricity loads")

# rescale the y-axis
ax.set_ylim([1.05*df_neg.sum(axis=1).min(), 1.05*df_pos.sum(axis=1).max()])
ax.legend(ncol=1, loc="upper center", bbox_to_anchor=(1.17, 1.01), title="Legend for left y-axis")
ax2.legend(title="Legend for right y-axis",  loc="upper right")
ax.set_ylabel("total electriyity balance [GW]")
ax2.set_ylabel("lmp  [€/MWh]")
ax.set_xlabel("")
ax.grid(True)
ax.set_title(f"Electricity balance from {start_date} until {end_date} ({model})", fontsize=16, pad=15,  **font1)
fig.tight_layout()

#plt.close()
plt.show()

fig.savefig(f"{PLOT_DIR}01_general/4.2_systems_technologies/el_bal_{datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d_%H_%M_%S')}__{datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d_%H_%M_%S')}_{model}.png")

In [ ]:
# Investigate in special case

# generation per hour
c ="battery discharger"
max(nb_el_h2.loc["2013-01-23 15:00:00" : "2013-01-25 00:00:00",c] / 3)

In [ ]:
# check if running peak plants are running on max cap for their location
# generation / capacity
(-n_no.links_t.p1.loc["2013-01-23 15:00:00" : "2013-01-25 00:00:00", n_no.links.carrier == "urban central solid biomass CHP CC"] / n_no.links.p_nom_opt[n_no.links.carrier == "urban central solid biomass CHP CC"]).head()

In [ ]:
n_no.links.efficiency[n_no.links.carrier == "urban central solid biomass CHP CC"].head()

max/min gen/con / average gen/con from from "2013-01-23 15:00:00" : "2013-01-25 00:00:00"

|technology   |STST   |EXP   |
|---|---|---|
|'gas CHP'   |143 / 135  |84 / 84 |   
|'biomas CHP CC'   | 26 / 26  |  20 / 20 |  
|'battery discharger'  |69 / 41   |32 /   15|  
|  'heat pump' | -138 /  -121| -124 / -111 |   
| 'resistive heater'  |-108/ -45  |-58 /  -130  
| 'PHS'    |18 / 10 |35 /  15|
| 'hydro'    |91 /  88|99/  96|

### Monthly Balance

In [ ]:
# get list of months
month_list = pd.period_range(start='2013-01-01', end='2013-12-31', freq='M')
month_list = [month.strftime("%b-%Y") for month in month_list]

res_no_all_pos = pd.DataFrame(index=month_list, columns= nb_el_no_all.columns)
res_no_all_neg = pd.DataFrame(index=month_list, columns= nb_el_no_all.columns)
res_h2_all_pos = pd.DataFrame(index=month_list, columns= nb_el_h2_all.columns)
res_h2_all_neg = pd.DataFrame(index=month_list, columns= nb_el_h2_all.columns)

iterables = [month_list, ["no H2 network", "H2 network"]]
multiindex = pd.MultiIndex.from_product(iterables, names=["month", "scenario"])
index_union = nb_el_no_all.columns.union(nb_el_h2_all.columns)
res_pos = pd.DataFrame(index=multiindex, columns=index_union)
res_neg = pd.DataFrame(index=multiindex, columns=index_union)

df_no_neg, df_no_pos = nb_el_no_all.clip(upper=0), nb_el_no_all.clip(lower=0)
df_h2_neg, df_h2_pos = nb_el_h2_all.clip(upper=0), nb_el_h2_all.clip(lower=0)

for month in month_list:
    res_no_all_pos.loc[month] = df_no_pos[month].sum()
    res_no_all_neg.loc[month] = df_no_neg[month].sum()
    res_h2_all_pos.loc[month] = df_h2_pos[month].sum()
    res_h2_all_neg.loc[month] = df_h2_neg[month].sum()

    res_pos.loc[month, "no H2 network"] = df_no_pos[month].sum()
    res_pos.loc[month, "H2 network"] = df_h2_pos[month].sum()
    res_neg.loc[month, "no H2 network"] = df_no_neg[month].sum()
    res_neg.loc[month, "H2 network"] = df_h2_neg[month].sum()

In [ ]:
df_no_pos

In [ ]:
# thesis plot (STST)

fig, (ax1, ax2) = plt.subplots(ncols=1, nrows=2, figsize=(20, 10))

# sort carriers by ouptut and exclude all with less than 40 GW
i_no_gen = res_no_all_pos.sum().sort_values(ascending=False)[res_no_all_pos.sum().sort_values(ascending=False) > 40 * 1e3].index

(res_no_all_pos[i_no_gen]/1000).plot(ax=ax1, kind="bar", stacked=True, grid=False, color=[carrier_colors[c] for c in res_no_all_pos[i_no_gen].columns])
ax1.legend(ncol=1, bbox_to_anchor=(1, 1))
ax1.set_title("Monthly distribution of electricity generation (STST)", fontsize=16, pad=15,  **font1)
ax1.set_ylabel("Generation in TWh")
ax1.set_xticks(ticks=ax1.get_xticks(), labels=[month[:3] for month in month_list], rotation=0)

# exclude solar and onwind
i_no_gen_ex = i_no_gen.drop(["onwind", "solar"])

(res_no_all_pos[i_no_gen_ex]/1000).plot(ax=ax2, kind="bar", stacked=True, grid=False, color=[carrier_colors[c] for c in res_pos[i_no_gen_ex].columns])
ax2.legend(ncol=1, bbox_to_anchor=(1, 1))
ax2.set_title("Monthly distribution of electricity generation without wind and solar (STST)", fontsize=16, pad=15,  **font1)
ax2.set_ylabel("Generation in TWh")
ax2.set_xticks(ticks=ax2.get_xticks(), labels=[month[:3] for month in month_list], rotation=0)

fig.tight_layout(pad=3)
fig.show()

#fig.savefig(f"{PLOT_DIR}01_general/4.2_systems_technologies/el_bal_monthly_STST.png")

In [ ]:
# Further investigation
res_no_all_pos[i_no_gen_ex]["gas CHP"].loc[["Jan-2013", "Feb-2013"]].sum() / (res_no_all_pos[i_no_gen_ex]["gas CHP"].sum() - res_no_all_pos[i_no_gen_ex]["gas CHP"].loc[["Jan-2013", "Feb-2013"]].sum())

In [ ]:
# EXP

fig, (ax1, ax2) = plt.subplots(ncols=1, nrows=2, figsize=(20, 10))

# sort carriers by ouptut and exclude all with less than 40 GW
i_h2_gen = res_h2_all_pos.sum().sort_values(ascending=False)[res_h2_all_pos.sum().sort_values(ascending=False) > 40 * 1e3].index

(res_h2_all_pos[i_no_gen]/1000).plot(ax=ax1, kind="bar", stacked=True, grid=False, color=[carrier_colors[c] for c in res_h2_all_pos[i_no_gen].columns])
ax1.legend(ncol=1, bbox_to_anchor=(1, 1))
ax1.set_title("Monthly distribution of electricity generation (EXP)", fontsize=16, pad=15,  **font1)
ax1.set_ylabel("Generation in TWh")
ax1.set_xticks(ticks=ax1.get_xticks(), labels=[month[:3] for month in month_list], rotation=0)
ax1.set_ylim([0,1000])

# exclude solar and onwind
i_h2_gen_ex = i_h2_gen.drop(["onwind", "solar"])

(res_h2_all_pos[i_h2_gen_ex]/1000).plot(ax=ax2, kind="bar", stacked=True, grid=False, color=[carrier_colors[c] for c in res_pos[i_h2_gen_ex].columns])
ax2.legend(ncol=1, bbox_to_anchor=(1, 1))
ax2.set_title("Monthly distribution of electricity generation without wind and solar (EXP)", fontsize=16, pad=15,  **font1)
ax2.set_ylabel("Generation in TWh")
ax2.set_xticks(ticks=ax2.get_xticks(), labels=[month[:3] for month in month_list], rotation=0)

fig.tight_layout(pad=3)
plt.close()
fig.show()

#fig.savefig(f"{PLOT_DIR}01_general/4.2_systems_technologies/el_bal_monthly_EXP.png")

In [ ]:
# both scenarios

fig, (ax1, ax2) = plt.subplots(ncols=1, nrows=2, figsize=(20, 12))

# sort carriers by ouptut and exclude all with less than 40 GWh
index = res_pos.sum().sort_values(ascending=False)[res_pos.sum().sort_values(ascending=False) > 40*1e3].index

res_pos[index].plot(ax=ax1, kind="bar", stacked=True, grid=False, color=[carrier_colors[c] for c in res_pos[index].columns], width=0.8)
ax1.legend(ncol=1, bbox_to_anchor=(1, 1))
ax1.set_title("Monthly distribution of electricity generation", fontsize=16, pad=15,  **font1)

# exclude solar and onwind
index = index.drop(["onwind", "solar"])

res_pos[index].plot(ax=ax2, kind="bar", stacked=True, grid=False, color=[carrier_colors[c] for c in res_pos[index].columns], width=0.8)
ax2.legend(ncol=1, bbox_to_anchor=(1, 1))
ax2.set_title("Monthly distribution of electricity generation without wind and solar", fontsize=16, pad=15,  **font1)

fig.tight_layout(pad=3)

plt.close()
fig.show()

In [ ]:
# consumption carriers
abs(res_no_all_neg.sum()).sort_values(ascending=False)[abs(res_no_all_neg.sum()).sort_values(ascending=False) > 1].index

In [ ]:
# STST

fig, (ax1, ax2) = plt.subplots(ncols=1, nrows=2, figsize=(20, 12))

# sort carriers by ouptut and exclude all with less than 1 GW
i_no_con = abs(res_no_all_neg.sum()).sort_values(ascending=False)[abs(res_no_all_neg.sum()).sort_values(ascending=False) > 1].index

res_no_all_neg[i_no_con].plot(ax=ax1, kind="bar", stacked=True, grid=False, color=[carrier_colors[c] for c in res_no_all_neg[i_no_con].columns])
ax1.legend(ncol=1, bbox_to_anchor=(1, 1))
ax1.set_title("Monthly distribution of electricity consumption (STST)", fontsize=16, pad=15,  **font1)

# exclude
i_no_con_ex = i_no_con.drop(["H2 Electrolysis", "BEV charger", "heat pump"])

res_no_all_neg[i_no_con_ex].plot(ax=ax2, kind="bar", stacked=True, grid=False, color=[carrier_colors[c] for c in res_no_all_neg[i_no_con_ex].columns])
ax2.legend(ncol=1, bbox_to_anchor=(1, 1))
ax2.set_title("Monthly distribution of electricity consumption without electrolysis, BEV charger and heat pump", fontsize=16, pad=15,  **font1)

fig.tight_layout(pad=3)
fig.show()

In [ ]:
# both

fig, (ax1, ax2) = plt.subplots(ncols=1, nrows=2, figsize=(20, 12))

# sort carriers by ouptut and exclude all with less than 1 GW
index = abs(res_neg.sum()).sort_values(ascending=False)[abs(res_neg.sum()).sort_values(ascending=False) > 1].index

res_neg[index].plot(ax=ax1, kind="bar", stacked=True, grid=False, color=[carrier_colors[c] for c in res_neg[index].columns], width=0.8)
ax1.legend(ncol=1, bbox_to_anchor=(1, 1))
ax1.set_title("Monthly distribution of electricity consumption", fontsize=16, pad=15,  **font1)

# exclude
index = index.drop(["H2 Electrolysis", "BEV charger"])

res_neg[index].plot(ax=ax2, kind="bar", stacked=True, grid=False, color=[carrier_colors[c] for c in res_neg[index].columns], width=0.8)
ax2.legend(ncol=1, bbox_to_anchor=(1, 1))
ax2.set_title("Monthly distribution of electricity consumption without electrolysis and BEV charger", fontsize=16, pad=15,  **font1)

fig.tight_layout(pad=3)
plt.close()
fig.show()

## Focus on technologies

### Duration curves

In [ ]:
# thesis plot
# Duration curves of electricity generating technologies

model = "STST"

if model == "STST":
    n = n_no
elif model == "EXP":
    n = n_h2

# reverse renaming to find all carriers in network
i_no_gen_ex_rev = [carrier_renaming_reverse.get(n, n) for n in i_no_gen_ex]
index = list(i_no_gen_ex_rev)

fig, (ax1, ax2) = plt.subplots(ncols=2, nrows=1, figsize=(16, 7))

for c in ["onwind", "solar"]:
    gen = (n.generators_t.p.loc[:, n.generators.carrier == c] / 1000)
    gen = pd.DataFrame(gen.sum(axis=1).sort_values(ascending=False)).set_index(pd.Index(np.linspace(0, 1, num=2920)))
    ax1.plot(gen, color = carrier_colors[c], label=c)

# plot solar + onwind
gen = (n.generators_t.p.loc[:, (n.generators.carrier == "onwind") | (n.generators.carrier == "solar")] / 1000)
gen = pd.DataFrame(gen.sum(axis=1).sort_values(ascending=False)).set_index(pd.Index(np.linspace(0, 1, num=2920)))
ax1.plot(gen, color = "black", label= "onwind + solar")

# plot load
load = (n.loads_t.p.loc[: , n.loads.carrier.isin(["electricity", "industry electricity", "agriculture electricity"])].sum(axis=1) / 1000) 
ax1.hlines(y=load.mean(), xmin=0, xmax=1, ls="dashed", linewidth=2, color='saddlebrown', label="electricity load (mean)")
ax1.hlines(y=load.max(), xmin=0, xmax=1, ls="dashed", linewidth=2, color='red', label = "electricity load (max)")

ax1.set_ylabel("generation [GW]")
ax1.set_xlabel("Fraction of total time")
ax1.set_facecolor("lightgrey")
ax1.legend()
ax1.grid(True)

for c in c_gen(index, n):
    if c == "":
        continue
    gen = (n.generators_t.p.loc[:, n.generators.carrier == c] / 1000)
    gen = pd.DataFrame(gen.sum(axis=1).sort_values(ascending=False)).set_index(pd.Index(np.linspace(0, 1, num=2920)))
    ax2.plot(gen, color = carrier_colors[c], label=c, marker=markers[index.index(c)], markevery=[0,int(0.25*2920), int(0.5*2920),int(0.75*2920)-1])

for c in c_link(index, n):
    gen = (-n.links_t.p1.loc[:, n.links.carrier == c] / 1000)
    gen = pd.DataFrame(gen.sum(axis=1).sort_values(ascending=False)).set_index(pd.Index(np.linspace(0, 1, num=2920)))
    ax2.plot(gen, color = carrier_colors[c], label=c.replace("urban central solid ", "").replace("urban central ", ""),marker=markers[index.index(c)], markevery=[0,int(0.25*2920), int(0.5*2920),int(0.75*2920)-1])

for c in c_su(index, n):
    gen = (n.storage_units_t.p.loc[:, n.storage_units.carrier == c] / 1000)
    gen = pd.DataFrame(gen.sum(axis=1).sort_values(ascending=False)).set_index(pd.Index(np.linspace(0, 1, num=2920)))
    ax2.plot(gen, color = carrier_colors[c], label=c, marker=markers[index.index(c)], markevery=[0,int(0.25*2920), int(0.5*2920),int(0.75*2920)-1])

ax2.set_ylabel("Generation [GW]")
ax2.set_xlabel("Fraction of total time")
ax2.set_facecolor("lightgrey")
ax2.legend(ncol=2)
ax2.grid(True)

fig.suptitle(f"Duration curves of electricity generating technologies ({model})", fontsize=20, **font1)
fig.tight_layout()
#plt.close()
plt.show()

fig.savefig(f"{PLOT_DIR}01_general/4.2_systems_technologies/el_gen_dur_{model}.png")

In [ ]:
# check if all el_gen are there:
set(c_el_gen_s).difference(ax1.get_legend_handles_labels()[1] + ax2.get_legend_handles_labels()[1])

In [ ]:
load = (n_no.loads_t.p.loc[: , n_no.loads.carrier.isin(["electricity", "industry electricity", "agriculture electricity"])].sum(axis=1) / 1000) * 3
plt.plot(load)
load.mean()

In [ ]:
# thesis plot
# Duration curves of electricity generating technologies

model = "EXP"

if model == "STST":
    n = n_no
    df = df_stst_ons
elif model == "EXP":
    n = n_h2
    df = df_exp_ons

# reverse renaming to find all carriers in network
i_no_gen_ex_rev = [carrier_renaming_reverse.get(n, n) for n in i_no_gen_ex]
index = list(i_no_gen_ex_rev)

fig, (ax1, ax2) = plt.subplots(ncols=2, nrows=1, figsize=(16, 7))

for c in ["onwind", "solar"]:
    gen = (n.generators_t.p.loc[:, n.generators.carrier == c] / 1000)
    gen = pd.DataFrame((gen.sum(axis=1)/df[f"{c}_cap_gen_el"].sum()).sort_values(ascending=False)).set_index(pd.Index(np.linspace(0, 1, num=2920)))
    ax1.plot(gen, color = carrier_colors[c], label=c)

ax1.set_ylabel("Generation in unit of maximum capacity (%)")
ax1.set_xlabel("Fraction of total time")
ax1.set_facecolor("lightgrey")
ax1.legend()
ax1.grid(True)

for c in c_gen(index, n):
    if c == "":
        continue
    gen = (n.generators_t.p.loc[:, n.generators.carrier == c] / 1000)
    gen = pd.DataFrame((gen.sum(axis=1)/df[f"{c}_cap_gen_el"].sum()).sort_values(ascending=False)).set_index(pd.Index(np.linspace(0, 1, num=2920)))
    ax2.plot(gen, color = carrier_colors[c], label=c, marker=markers[index.index(c)], markevery=[0,int(0.25*2920), int(0.5*2920),int(0.75*2920)-1])

for c in c_link(index, n):
    gen = (-n.links_t.p1.loc[:, n.links.carrier == c] / 1000)
    gen = pd.DataFrame((gen.sum(axis=1)/df[f"{c}_cap_gen_el"].sum()).sort_values(ascending=False)).set_index(pd.Index(np.linspace(0, 1, num=2920)))
    ax2.plot(gen, color = carrier_colors[c], label=c.replace("urban central solid ", "").replace("urban central ", ""),marker=markers[index.index(c)], markevery=[0,int(0.25*2920), int(0.5*2920),int(0.75*2920)-1])

for c in c_su(index, n):
    gen = (n.storage_units_t.p.loc[:, n.storage_units.carrier == c] / 1000)
    gen = pd.DataFrame((gen.sum(axis=1)/df[f"{c}_cap_gen_el"].sum()).sort_values(ascending=False)).set_index(pd.Index(np.linspace(0, 1, num=2920)))
    ax2.plot(gen, color = carrier_colors[c], label=c, marker=markers[index.index(c)], markevery=[0,int(0.25*2920), int(0.5*2920),int(0.75*2920)-1])

ax2.set_ylabel("Generation in unit of maximum capacity (%)")
ax2.set_xlabel("Fraction of total time")
ax2.set_facecolor("lightgrey")
ax2.legend(ncol=2)
ax2.grid(True)

fig.suptitle(f"Normalised duration curves of electricity generating technologies ({model})", fontsize=20, **font1)
fig.tight_layout()
#plt.close()
plt.show()

fig.savefig(f"{PLOT_DIR}01_general/4.2_systems_technologies/el_gen_dur_{model}_normalised.png")

In [ ]:
c = "hydro"
gen = (n.storage_units_t.p.loc[:, n.storage_units.carrier == c] / 1000)
gen = pd.DataFrame((gen.sum(axis=1)).sort_values(ascending=False)).set_index(pd.Index(np.linspace(0, 1, num=2920)))
gen

In [ ]:
n.storage_units.p_nom_opt[n.storage_units.carrier == c].sum() / 1e3

In [ ]:
carrier_colors = {
    'AC': 'dimgrey',
    'BEV charger': 'lightyellow',
    'DAC': 'darkgrey',
    'DC': 'lightgrey',
    'Fischer-Tropsch': 'gold',
    'H2': 'turquoise',
    'H2 Electrolysis': 'fuchsia',
    'H2 Fuel Cell': 'indigo',
    'H2 for industry': 'cadetblue',
    'H2 for shipping': 'navy',
    'H2 liquefaction': 'lightskyblue',
    'Li ion': 'greenyellow',
    'low-temperature heat for industry': "lightsalmon",
    'OCGT': 'saddlebrown',
    'PHS': 'dodgerblue',
    'SMR': 'darkseagreen',
    'SMR CC': 'palegreen',
    'Sabatier': 'peachpuff',
    'V2G': 'honeydew',
    'agriculture electricity': 'goldenrod',
    'agriculture heat': 'peru',
    'battery': 'hotpink',
    'battery charger': 'darkorange',
    'battery discharger': 'thistle',
    'electricity': 'palegoldenrod',
    'electricity distribution grid': 'thistle',
    'gas': 'orange',
    'home battery': 'violet',
    'home battery charger': 'blueviolet',
    'home battery discharger': 'plum',
    'hydro': 'cornflowerblue',
    'industry electricity': 'rosybrown',
    'land transport EV': 'chocolate',
    'land transport fuel cell': 'peru',
    'offwind-ac': 'cyan',
    'offwind-dc': 'maroon',
    'oil': 'k',
    'onwind': 'green',
    'residential rural gas boiler': 'gold',
    'residential rural ground heat pump': 'darkred',
    'residential rural heat': 'rosybrown',
    'residential rural resistive heater': 'lightgreen',
    'residential rural water tanks charger': 'cadetblue',
    'residential rural water tanks discharger': 'palevioletred',
    'residential urban decentral air heat pump': 'firebrick',
    'residential urban decentral gas boiler': 'lemonchiffon',
    'residential urban decentral heat': 'lightcoral',
    'residential urban decentral resistive heater': 'lightseagreen',
    'residential urban decentral solar thermal': 'salmon',
    'residential urban decentral water tanks charger': 'powderblue',
    'residential urban decentral water tanks discharger': 'crimson',
    'ror': 'blue',
    'services rural gas boiler': 'khaki',
    'services rural ground heat pump': 'maroon',
    'services rural heat': 'indianred',
    'services rural resistive heater': 'seagreen',
    'services rural water tanks charger': 'lightblue',
    'services rural water tanks discharger': 'pink',
    'services urban decentral air heat pump': 'orangered',
    'services urban decentral gas boiler': 'palegoldenrod',
    'services urban decentral heat': 'brown',
    'services urban decentral resistive heater': 'mediumspringgreen',
    'services urban decentral solar thermal': 'tomato',
    'services urban decentral water tanks charger': 'deepskyblue',
    'services urban decentral water tanks discharger': 'lightpink',
    'solar': 'yellow',
    'solar rooftop': 'brown',
    'urban central air heat pump': 'red',
    'urban central gas CHP': 'darkorange',
    'gas CHP': 'darkorange',
    'urban central gas CHP CC': 'navajowhite',
    'gas CHP CC': 'navajowhite',
    'urban central gas boiler': 'darkkhaki',
    'urban central heat': 'firebrick',
    'urban central resistive heater': 'lime',
    'urban central solar thermal': 'pink',
    'urban central solid biomass CHP CC': 'navajowhite',
    'biomass CHP CC': 'navajowhite',
    'urban central solid biomass CHP': 'sandybrown',
    'biomass CHP': 'sandybrown',
    'urban central water tanks charger': 'skyblue',
    'urban central water tanks discharger': 'mediumvioletred',
    'resistive heater': 'seagreen',
    'gas boiler': 'khaki',
    'heat pump': 'darkred',
    'water tanks charger': 'cadetblue',
    'water tanks discharger': 'palevioletred',
    'solar thermal': 'tomato',
    'H2 pipeline': 'pink',
    'H2 pipeline retrofitted': 'violet',
}

In [ ]:
# Duration curves of electricity consuming technologies

model = "EXP"

if model == "STST":
    n = n_no
elif model == "EXP":
    n = n_h2

# reverse renaming to find all carriers in network
i_no_gen_ex_rev = [carrier_renaming_reverse.get(n, n) for n in i_no_gen_ex]
index = list(i_no_gen_ex_rev)


fig, (ax1, ax2) = plt.subplots(ncols=2, nrows=1, figsize=(16, 7))

index = ["H2 Electrolysis", "BEV charger"]  #heat_pump # heat pump is missing due to grouping

for c in c_gen(index, n_no):
    gen = n.generators_t.p.loc[:, n.generators.carrier == c] / 1000
    gen = pd.DataFrame(gen.sum(axis=1).sort_values(ascending=True)).set_index(pd.Index(np.linspace(0, 1, num=2920)))
    ax1.plot(gen, color = carrier_colors[c], label=c)

for c in c_link(index, n_no):
    gen = -n.links_t.p0.loc[:, n.links.carrier == c] / 1000 
    gen = pd.DataFrame(gen.sum(axis=1).sort_values(ascending=True)).set_index(pd.Index(np.linspace(0, 1, num=2920)))
    ax1.plot(gen, color = carrier_colors[c], label=c.replace("urban central solid ", "").replace("urban central ", ""), marker=markers[index.index(c)], markevery=[0,int(0.25*2920), int(0.5*2920),int(0.75*2920)-1])

ax1.set_ylabel("Consumption [GW]")
ax1.set_xlabel("Fraction of total time")
ax1.legend(loc="lower right")
ax1.set_facecolor("lightgrey")
ax1.grid(True)

index = list(i_no_con_ex) + ["urban central resistive heater"]+ ["urban central air heat pump"] #resistive_heater # resistive heater is missing due to grouping
# remove
if 'DAC' in index: index.remove('DAC')
if 'home battery charger' in index: index.remove('home battery charger')

for c in c_gen(index, n_no):
    gen = n.generators_t.p.loc[:, n.generators.carrier == c] / 1000 
    gen = pd.DataFrame(gen.sum(axis=1).sort_values(ascending=Ture)).set_index(pd.Index(np.linspace(0, 1, num=2920)))
    ax2.plot(gen, color = carrier_colors[c], label=c, marker=markers[index.index(c)], markevery=[0,int(0.25*2920), int(0.5*2920),int(0.75*2920)-1])

for c in c_link(index, n_no):
    gen = -n.links_t.p0.loc[:, n.links.carrier == c] / 1000 
    gen = pd.DataFrame(gen.sum(axis=1).sort_values(ascending=True)).set_index(pd.Index(np.linspace(0, 1, num=2920)))
    ax2.plot(gen, color = carrier_colors[c], label=c.replace("urban central solid ", "").replace("urban central ", ""), marker=markers[index.index(c)], markevery=[0,int(0.25*2920), int(0.5*2920),int(0.75*2920)-1])

for c in c_su(index, n_no):
    gen = n.storage_units_t.p.loc[:, n.storage_units.carrier == c] / 1000 
    gen = pd.DataFrame(gen.sum(axis=1).sort_values(ascending=True)).set_index(pd.Index(np.linspace(0, 1, num=2920)))
    ax2.plot(gen, color = carrier_colors[c], label=c, marker=markers[index.index(c)], markevery=[0,int(0.25*2920), int(0.5*2920),int(0.75*2920)-1])

ax2.set_ylabel("Consumption [GW]")
ax2.set_xlabel("Fraction of total time")
ax2.legend(loc="lower right", ncol=1)
ax2.set_facecolor("lightgrey")
ax2.grid(True)

fig.suptitle(f"Duration curves of electricity consuming technologies ({model})", fontsize=20, **font1)
fig.tight_layout()
# plt.close()
plt.show()

fig.savefig(f"{PLOT_DIR}01_general/4.2_systems_technologies/el_con_dur_{model}.png")

Questions:
- Does it make more sense for the consuming carriers to look at p0 or p1? just difference in magnitude and sign -> probably p0 as this is the electricity consumption (for electricity generating links you also look at p1 which is the electricity generating bus


In [ ]:
# load duration curves of electricity consuming technologies

model = "EXP"

if model == "STST":
    n = n_no
    df = df_stst_ons
elif model == "EXP":
    n = n_h2
    df = df_exp_ons

# reverse renaming to find all carriers in network
i_no_gen_ex_rev = [carrier_renaming_reverse.get(n, n) for n in i_no_gen_ex]
index = list(i_no_gen_ex_rev)


fig, (ax1, ax2) = plt.subplots(ncols=2, nrows=1, figsize=(16, 7))

index = ["H2 Electrolysis", "BEV charger"]  #heat_pump # heat pump is missing due to grouping

for c in c_gen(index, n_no):
    gen = n.generators_t.p.loc[:, n.generators.carrier == c] / 1000
    gen = pd.DataFrame(gen.sum(axis=1).sort_values(ascending=True)).set_index(pd.Index(np.linspace(0, 1, num=2920)))
    ax1.plot(gen, color = carrier_colors[c], label=c)

for c in c_link(index, n_no):
    gen = -n.links_t.p0.loc[:, n.links.carrier == c] / 1000 
    gen = pd.DataFrame((gen.sum(axis=1)/df[f"{c}_cap_con_el"].sum()).sort_values(ascending=True)).set_index(pd.Index(np.linspace(0, 1, num=2920)))
    ax1.plot(gen, color = carrier_colors[c], label=c.replace("urban central solid ", "").replace("urban central ", ""), marker=markers[index.index(c)], markevery=[0,int(0.25*2920), int(0.5*2920),int(0.75*2920)-1])

ax1.set_ylabel("Consumption in unit of maximum capacity (%)")
ax1.set_xlabel("Fraction of total time")
ax1.legend(loc="lower right")
ax1.set_facecolor("lightgrey")
ax1.grid(True)

index = list(i_no_con_ex) + ["urban central resistive heater"]+ ["urban central air heat pump"] #resistive_heater # resistive heater is missing due to grouping
# remove
if 'DAC' in index: index.remove('DAC')
if 'home battery charger' in index: index.remove('home battery charger')

for c in c_gen(index, n_no):
    gen = n.generators_t.p.loc[:, n.generators.carrier == c] / 1000 
    gen = pd.DataFrame((gen.sum(axis=1)/df[f"{c}_cap_con_el"].sum()).sort_values(ascending=Ture)).set_index(pd.Index(np.linspace(0, 1, num=2920)))
    ax2.plot(gen, color = carrier_colors[c], label=c, marker=markers[index.index(c)], markevery=[0,int(0.25*2920), int(0.5*2920),int(0.75*2920)-1])

for c in c_link(index, n_no):
    gen = -n.links_t.p0.loc[:, n.links.carrier == c] / 1000 
    gen = pd.DataFrame((gen.sum(axis=1)/df[f"{c}_cap_con_el"].sum()).sort_values(ascending=True)).set_index(pd.Index(np.linspace(0, 1, num=2920)))
    ax2.plot(gen, color = carrier_colors[c], label=c.replace("urban central solid ", "").replace("urban central ", ""), marker=markers[index.index(c)], markevery=[0,int(0.25*2920), int(0.5*2920),int(0.75*2920)-1])

for c in c_su(index, n_no):
    gen = n.storage_units_t.p.loc[:, n.storage_units.carrier == c] / 1000 
    gen = pd.DataFrame((gen.sum(axis=1)/df[f"{c}_cap_con_el"].sum()).sort_values(ascending=True)).set_index(pd.Index(np.linspace(0, 1, num=2920)))
    ax2.plot(gen, color = carrier_colors[c], label=c, marker=markers[index.index(c)], markevery=[0,int(0.25*2920), int(0.5*2920),int(0.75*2920)-1])

ax2.set_ylabel("Consumption in unit of maximum capacity (%)")
ax2.set_xlabel("Fraction of total time")
ax2.legend(loc="lower right", ncol=1)
ax2.set_facecolor("lightgrey")
ax2.grid(True)

fig.suptitle(f"Normalised duration curves of electricity consuming technologies ({model})", fontsize=20, **font1)
fig.tight_layout()
# plt.close()
plt.show()

fig.savefig(f"{PLOT_DIR}01_general/4.2_systems_technologies/el_con_dur_{model}_normalised.png")

### Capacity factors

**Question:**
- You have to aggregate over two dimensions: space and time
- Which dimension makes more sense to aggregate over
- You could visualize space coponent via map or boxplot
- You could visualize time component via heatmap

In [ ]:
# capacity factors across regions as boxplot (STST & EXP)

# boxplot propertes
medianprops = dict(color="black",linewidth=1.5)
meanprops = {"marker":"d","markerfacecolor":"white", "markeredgecolor":"black"}
flierprops= {'marker': 'x', 'markersize': 5, 'markeredgecolor': 'black'}
whiskerprops = dict(linestyle='-',linewidth=1.0, color='black')

# data
stst_el_gen = df_stst_ons[[c + "_cf" for c in c_el_gen_s]]
stst_el_gen = stst_el_gen.values
exp_el_gen = df_exp_ons[[c + "_cf" for c in c_el_gen_s]]
exp_el_gen = exp_el_gen.values

# Filter data using np.isnan
mask_stst = ~np.isnan(stst_el_gen)
filtered_stst = [d[m] for d, m in zip(stst_el_gen.T, mask_stst.T)]
mask_exp = ~np.isnan(exp_el_gen)
filtered_exp = [d[m] for d, m in zip(exp_el_gen.T, mask_exp.T)]

ticks = [carrier_renaming.get(n, n) for n in c_el_gen_s]
index = c_el_gen_s
fig, ax = plt.subplots(figsize=(12, 6))

stst_plot = plt.boxplot(filtered_stst,
                        positions=np.array(np.arange(len(ticks)))*2.0-0.35,
                        widths=0.6,
                        patch_artist=True,
                        showmeans=True,
                        meanprops=meanprops,
                        medianprops=medianprops,
                        flierprops=flierprops,
                        whiskerprops=whiskerprops,
                        zorder=1
                        )

exp_plot = plt.boxplot(filtered_exp,
                       positions=np.array(np.arange(len(ticks)))*2.0+0.35,
                       widths=0.6,
                       patch_artist=True,
                       showmeans=True,
                       meanprops=meanprops,
                       medianprops=medianprops,
                       flierprops=flierprops,
                       whiskerprops=whiskerprops,
                       zorder=2
                       )

# generation weighted mean
gwm_cf_stst =np.multiply(df_stst_ons[[c + "_cf" for c in c_el_gen_s]], (df_stst_ons[[c + "_gen" for c in c_el_gen_s]] / df_stst_ons[[c + "_gen" for c in c_el_gen_s]].sum())).sum()
gwm_cf_exp =np.multiply(df_exp_ons[[c + "_cf" for c in c_el_gen_s]], (df_exp_ons[[c + "_gen" for c in c_el_gen_s]] / df_exp_ons[[c + "_gen" for c in c_el_gen_s]].sum())).sum()
ax.plot(np.array(np.arange(len(ticks)))*2.0-0.35, gwm_cf_stst.transpose(),"x", marker='*', color="red", markersize= 10, markerfacecolor="white", zorder=3)
ax.plot(np.array(np.arange(len(ticks)))*2.0+0.35, gwm_cf_exp.transpose(),"x", marker='*', color="red", markersize= 10, markerfacecolor="white",zorder=4)


for box, col in zip(stst_plot['boxes'],[carrier_colors[c] for c in index]):
    # change outline color
    box.set_facecolor(col)
    box.set_linestyle('--')

for box, col in zip(exp_plot['boxes'],[carrier_colors[c] for c in index]):
    # change outline color
    box.set_facecolor(col)

# sample sizes
for i, sample_size in enumerate(df_stst_ons[[f"{i}_cf" for i in index]].count()):
    ax.annotate(sample_size, xy=(0,0),  xycoords='axes fraction',
        xytext=((i+0.35)/len(index),1.01), textcoords='axes fraction', color="blue")

#for i, sample_size in enumerate(df_exp_ons[[f"{i}_cf" for i in index]].count()):
#    ax.annotate(sample_size, xy=(0,0),  xycoords='axes fraction',
#        xytext=((i+0.55)/len(index),1), textcoords='axes fraction', color="red")

# explanations
plt.xticks(np.arange(0, len(ticks) * 2, 2), ticks)
plt.xticks(rotation=90)
# plt.title("Market values of electricity generating technologies across the regions (STST vs. EXP)", fontsize=16, pad=20,  **font1)

# cosmetics
ax.patch.set_facecolor('lightgrey')
ax.patch.set_alpha(0.5)
ax.grid(True)
ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
ax.set_ylabel("capacity factor [%]")

# legend
patch1 = matplotlib.patches.Patch(ls="--", facecolor="white", edgecolor="black")
patch2 = matplotlib.patches.Patch(ls="-", facecolor="white", edgecolor="black")
ax.legend([patch1, patch2], ['STST', 'EXP'], loc="upper left")

fig.tight_layout()
plt.show()

fig.savefig(f"{PLOT_DIR}01_general/4.2_systems_technologies/el_gen_cap_fac_STST_EXP.png")

In [ ]:
# Further analysis
c = "onwind"
df_exp_ons.loc[:, [f"{c}_cf", f"{c}_gen"]].sort_values(by=f"{c}_gen", ascending=False).head(10).mean()

In [ ]:
# Norway analysis
i_no = df_stst_ons.index[df_stst_ons.index.str.contains("NO")]
df_exp_ons.loc[i_no, "hydro_cf"]

In [ ]:
i_no = df_stst_ons.index[df_stst_ons.index.str.contains("NO")]
df_stst_ons.loc[i_no, "hydro_cf"]

In [ ]:
df_stst_ons[[c + "_cf" for c in c_el_gen_s]].describe()

In [ ]:
df_exp_ons[[c + "_cf" for c in c_el_gen_s]].describe()

In [ ]:
# thesis plot
# capacity factors across regions as map VRE

carriers = ["onwind", "solar", "offwind-dc", "ror"]
model = "STST"

fig, axs = plt.subplots(ncols=2, nrows=2, subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(15, 12))
crs = ccrs.EqualEarth()

for i, ax in enumerate(axs.reshape(-1)):
    
    if model == "STST":
        if carriers[i] in ["offwind-dc", "offwind-ac"]:
            df = df_no_off
        else:
            df = df_no_ons
            
    elif model == "EXP":
        if carriers[i] in ["offwind-dc", "offwind-ac"]:
            df = df_h2_off
        else:
            df = df_h2_ons

    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')

    df.to_crs(crs.proj4_init).plot(column=f"{carriers[i]}_cf",
                                                   ax=ax,
                                                   cmap=plt.get_cmap("magma_r"),
                                                   linewidth=0.05,
                                                   edgecolor = 'grey',
                                                   legend=True,
                                                   legend_kwds={'label':"capacity factors",
                            'orientation': "vertical",
                                      'shrink' : 0.9})

    max_size = df[f"{carriers[i]}_gen"].max()
    df.to_crs(crs.proj4_init).centroid.plot(ax=ax, sizes=df[f"{carriers[i]}_gen"] / max_size * 200,  color="black", edgecolor="white")
    pypsa.plot.add_legend_circles(ax=ax, sizes=[0.6], labels=["Generation magnitude"], patch_kw={'color': 'black', 'edgecolor': 'white'}, legend_kw={'loc': 'upper left'})

    # always select same section
    xmin, ymin, xmax, ymax = df_no_off.to_crs(crs.proj4_init).total_bounds
    pad = 1 * 1e5  # add a padding around the geometry
    ax.set_xlim(xmin-pad, xmax+pad)
    ax.set_ylim(ymin-pad, ymax+pad)
    
    
    ax.set_title(f"{carriers[i]}", fontsize=18, **font1)

# fig.suptitle(f"Spatial Differences in the electricity generation of the VRE technologies ({model})", fontsize=16, **font1)
fig.tight_layout(pad=1)
plt.show()

fig.savefig(f"{PLOT_DIR}01_general/4.2_systems_technologies/cap_fac_el_gen_vre_{model}.png")

In [ ]:
carrier = "onwind"
# correlation capacity factors and generation
df_no_ons[f"{carrier}_cf"].corr(df_no_ons[f"{carrier}_gen"])

In [ ]:
# correlation with latitude
latitudes = pd.DataFrame([Point(coords).y for coords in df_no_ons['coords']], index=df_no_ons.index)
df_no_ons[f"{carrier}_cf"].corr(latitudes[0])

In [ ]:
df_no_ons[f"{carrier}_gen"].sort_values(ascending=False)

In [ ]:
from sklearn import preprocessing

sns.distplot(preprocessing.normalize(df_no_ons["solar_cf"].dropna().values.reshape(1, -1)), hist=False, rug=True, label="solar")
sns.distplot(preprocessing.normalize(df_no_ons["onwind_cf"].dropna().values.reshape(1, -1)), hist=False, rug=True, label="onwind")

plt.legend()
plt.show()

In [ ]:
# capacity factors across regions as map
carriers = ["offwind-dc", "offwind-ac"]

fig, axs = plt.subplots(ncols=2, nrows=1, subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(18, 16))
crs = ccrs.EqualEarth()

for i, ax in enumerate(axs.reshape(-1)):

    if carriers[i] in ["offwind-dc", "offwind-ac"]:
        df = df_no_off
    else:
        df = df_no_ons

    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')

    df.to_crs(crs.proj4_init).plot(column=f"{carriers[i]}_cf",
                                                   ax=ax,
                                                   cmap=plt.get_cmap("magma_r"),
                                                   linewidth=0.05,
                                                   edgecolor = 'grey',
                                                   legend=True,
                                                   legend_kwds={'label':"capacity factors",
                            'orientation': "vertical",
                                      'shrink' : 0.4})

    max_size = df[f"{carriers[i]}_gen"].max()
    df.to_crs(crs.proj4_init).centroid.plot(ax=ax, sizes=df[f"{carriers[i]}_gen"] / max_size *300,  color="black", edgecolor="white")
    pypsa.plot.add_legend_circles(ax=ax, sizes=[0.6], labels=["Generation magnitude"], patch_kw={'color': 'black', 'edgecolor': 'white'}, legend_kw={'loc': 'upper left', 'facecolor': 'lightgrey'})

    ax.set_title(f"{carriers[i]}", fontsize=16, **font1)
fig.tight_layout()
plt.show()

# Backup
#m_no.plot(ax=ax, projection=ccrs.EqualEarth(), bus_colors="black", bus_sizes=df[f"{carriers[i]}_gen"] / max_size, line_widths=0, link_widths=0)

In [ ]:
# thesis plot
# capacity factors across regions as map: Storage

carriers = ["battery discharger", "hydro", "V2G", "PHS"]
model = "STST"

if model == "STST":
    df = df_no_ons

elif model == "EXP":
    df = df_h2_ons

fig, axs = plt.subplots(ncols=2, nrows=2, subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(15, 12))
crs = ccrs.EqualEarth()

for i, ax in enumerate(axs.reshape(-1)):

    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')

    df.to_crs(crs.proj4_init).plot(column=f"{carriers[i]}_cf",
                                                   ax=ax,
                                                   cmap=plt.get_cmap("magma_r"),
                                                   linewidth=0.05,
                                                   edgecolor = 'grey',
                                                   legend=True,
                                                   legend_kwds={'label':"capacity factors",
                            'orientation': "vertical",
                                      'shrink' : 0.9})

    max_size = df[f"{carriers[i]}_gen"].max()
    df.to_crs(crs.proj4_init).centroid.plot(ax=ax, sizes=df[f"{carriers[i]}_gen"] / max_size *200,  color="black", edgecolor="white")
    pypsa.plot.add_legend_circles(ax=ax, sizes=[0.6], labels=["Generation magnitude"], patch_kw={'color': 'black', 'edgecolor': 'white'}, legend_kw={'loc': 'upper left'})

     # always select same section
    xmin, ymin, xmax, ymax = df_no_ons.to_crs(crs.proj4_init).total_bounds
    pad = 1 * 1e5  # add a padding around the geometry
    ax.set_xlim(xmin-pad, xmax+pad)
    ax.set_ylim(ymin-pad, ymax+pad)
    
    ax.set_title(f"{carriers[i]}", fontsize=18, **font1)

# fig.suptitle(f"Spatial Differences in the electricity generation of the VRE technologies ({model})", fontsize=16, **font1)
fig.tight_layout()
plt.show()

fig.savefig(f"{PLOT_DIR}01_general/4.2_systems_technologies/cap_fac_el_gen_store_{model}.png")

In [ ]:
carrier = "home battery discharger"
df_no_ons[f"{carrier}_gen"].describe()

In [ ]:
0.385120 / 0.182673

In [ ]:
# generation weighted capacity factor
carrier = "home battery discharger"
((df_no_ons[f"{carrier}_gen"] / df_no_ons[f"{carrier}_gen"].sum()) * df_no_ons[f"{carrier}_cf"]).sum()

In [ ]:
# Norway indices
no_i = df_no_ons[f"{carrier}_gen"].index[df_no_ons[f"{carrier}_gen"].index.str.contains('NO')]

carrier = "PHS"
df_no_ons[f"{carrier}_gen"].sort_values(ascending=False)

In [ ]:
df_no_ons[f"{carrier}_gen"][no_i].sum() / df_no_ons[f"{carrier}_gen"].sum()

In [ ]:
# thesis plot
# Difference of battery discharger

carriers = ["battery discharger", "battery discharger"]


fig, axs = plt.subplots(ncols=2, nrows=1, subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(15, 6))
crs = ccrs.EqualEarth()

for i, ax in enumerate(axs.reshape(-1)):
            
    if i == 0:
            df = df_stst_ons
            model = "STST"

    elif i == 1:
            df = df_exp_ons
            model = "EXP"

    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')

    df.to_crs(crs.proj4_init).plot(column=f"{carriers[i]}_cap",
                                                   ax=ax,
                                                   cmap=plt.get_cmap("magma_r"),
                                                   linewidth=0.05,
                                                   edgecolor = 'grey',
                                                   legend=True,
                                                   legend_kwds={'label':"capacity [GW]",
                            'orientation': "vertical",
                                      'shrink' : 0.9})

    max_size = df[f"{carriers[i]}_gen"].max()
    df.to_crs(crs.proj4_init).centroid.plot(ax=ax, sizes=df[f"{carriers[i]}_gen"] / max_size *200,  color="black", edgecolor="white")
    pypsa.plot.add_legend_circles(ax=ax, sizes=[0.6], labels=["Generation magnitude"], patch_kw={'color': 'black', 'edgecolor': 'white'}, legend_kw={'loc': 'upper left'})

     # always select same section
    xmin, ymin, xmax, ymax = df_stst_ons.to_crs(crs.proj4_init).total_bounds
    pad = 1 * 1e5  # add a padding around the geometry
    ax.set_xlim(xmin-pad, xmax+pad)
    ax.set_ylim(ymin-pad, ymax+pad)
    
    ax.set_title(f"{carriers[i]} ({model})", fontsize=18, **font1)

fig.tight_layout()
plt.show()

fig.savefig(f"{PLOT_DIR}01_general/4.2_systems_technologies/cap_fac_el_gen_battery_discharger_STST_EXP.png")

In [ ]:
c = "battery discharger"
df_stst_ons[f"{c}_cap"].sort_values(ascending=False).head(30)

In [ ]:
# thesis plot
# capacity factors across regions as map: Peak plants

carriers = ["urban central gas CHP", "urban central solid biomass CHP CC" ]
model = "STST"

if model == "STST":
    df = df_no_ons

elif model == "EXP":
    df = df_h2_ons

fig, axs = plt.subplots(ncols=2, nrows=1, subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(15, 6))
crs = ccrs.EqualEarth()

for i, ax in enumerate(axs):
    
    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')

    df.to_crs(crs.proj4_init).plot(column=f"{carriers[i]}_cf",
                                                   ax=ax,
                                                   cmap=plt.get_cmap("magma_r"),
                                                   linewidth=0.05,
                                                   edgecolor = 'grey',
                                                   legend=True,
                                                   legend_kwds={'label':"capacity factors", 'orientation': "vertical", 'shrink' : 0.9}
                                   )

    max_size = df[f"{carriers[i]}_gen"].max()
    df.to_crs(crs.proj4_init).centroid.plot(ax=ax, sizes=df[f"{carriers[i]}_gen"] / max_size *200,  color="black", edgecolor="white")
    pypsa.plot.add_legend_circles(ax=ax, sizes=[0.6], labels=["Generation magnitude"], patch_kw={'color': 'black', 'edgecolor': 'white'}, legend_kw={'loc': 'upper left', 'facecolor': 'lightgrey'})

    # always select same section
    xmin, ymin, xmax, ymax = df_no_ons.to_crs(crs.proj4_init).total_bounds
    pad = 1 * 1e5  # add a padding around the geometry
    ax.set_xlim(xmin-pad, xmax+pad)
    ax.set_ylim(ymin-pad, ymax+pad)
    
    ax.set_title(carriers[i].replace("urban central solid ", "").replace("urban central ", ""), fontsize=18, **font1)

fig.tight_layout()
plt.show()

fig.savefig(f"{PLOT_DIR}01_general/4.2_systems_technologies/cap_fac_el_gen_peak_{model}.png")

In [ ]:
# generation weighted capacity factor
carrier = "urban central gas CHP" #urban central solid biomass CHP CC' #"urban central gas CHP"
((df_no_ons[f"{carrier}_gen"] / df_no_ons[f"{carrier}_gen"].sum()) * df_no_ons[f"{carrier}_cf"]).sum()

In [ ]:
# generation through time as heatmap

carrier = "onwind"

# get generation in TWh
df = pd.DataFrame(n.generators_t.p.loc[:, n.generators.carrier == carrier].sum(axis=1)) / 1000

hours = df.index.hour.unique()[::-1]
df_start = pd.DataFrame(index=pd.Index(df.index.date).unique())

for hour in hours:
    df_start[str(hour)] = df[df.index.hour==hour].values

plt.figure(figsize=(12, 5))
ax = sns.heatmap(df_start.transpose(),
                 cmap=plt.get_cmap("magma_r"),
                 linewidth=0.001,
                 xticklabels=15,
                 cbar_kws={'label': 'generation in TWh'})

plt.title(f"{carrier}", fontsize=16, **font1)
plt.ylabel("hour of the day", fontsize=12, **font1)
plt.xlabel("day of the year", fontsize=12, **font1)

# Rewrite the y labels
x_labels = ax.get_xticks()
ax.xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%d-%b'))

plt.show()

In [ ]:
#TODO: you could calc capacity factors for every time step and also plot them as heatmap

### Capacity and Generation difference

In [ ]:
# VRE capacity and generation difference
# ["onwind", "solar", "offwind-dc", "ror"]
#["onwind", "ror", "solar", "solar rooftop", "offwind-dc", "offwind-ac"]

carriers = ["onwind", "ror", "solar", "solar rooftop", "offwind-dc", "offwind-ac"]

# STST - EXP
for c in carriers:
    # if one mv is nan the result is nan as well
    if c in ["offwind-dc", "offwind-ac"]:
        df_stst_off[f"{c}_cap_STST-EXP"] = df_stst_off[f"{c}_cap"] - df_exp_off[f"{c}_cap"]
    else:
        df_stst_ons[f"{c}_cap_STST-EXP"] = df_stst_ons[f"{c}_cap"] - df_exp_ons[f"{c}_cap"]

fig, axs = plt.subplots(ncols=2, nrows=int(len(carriers)/2), subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(15, len(carriers)/2*6))
crs = ccrs.EqualEarth()

for i, ax in enumerate(axs.reshape(-1)):

    if carriers[i] in ["offwind-dc", "offwind-ac"]:
        df = df_stst_off
        df_2 = df_exp_off
    else:
        df = df_stst_ons
        df_2 = df_exp_ons
        
    if carriers[i] == "ror":
        ax.axis('off')
        continue

    abs_max = max(abs(df[f"{carriers[i]}_cap_STST-EXP"].max()) , abs(df[f"{carriers[i]}_cap_STST-EXP"].min()))
    #unit_cap = "TW" if abs_max > 1e4 else "GW"
    #df[f"{carriers[i]}_cap_STST-EXP"] = (df[f"{carriers[i]}_cap_STST-EXP"] / 1e3) if abs_max > 1e4 else df[f"{carriers[i]}_cap_STST-EXP"]
    #abs_max = abs_max / 1e3 if abs_max > 1e4 else abs_max
    
    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')

    df.to_crs(crs.proj4_init).plot(column=f"{carriers[i]}_cap_STST-EXP",
                                   ax=ax,
                                   cmap=plt.get_cmap('RdYlGn'),
                                   vmax=abs_max,
                                   vmin=-abs_max,
                                   linewidth=0.05,
                                   edgecolor = 'grey',
                                   legend=True,
                                   legend_kwds={'label':f"Capacity differences (GW)",'orientation': "vertical",'shrink' : 0.9}
                                   )

    # difference in generation (STST-EXP)
    gen_diff = df[f"{carriers[i]}_gen"] - df_2[f"{carriers[i]}_gen"]
    # red if negative and green if positive
    colors = ['red' if (x < 0) else 'green' for x in gen_diff ]

    max_size = abs(gen_diff).max()
    df.to_crs(crs.proj4_init).centroid.plot(ax=ax, sizes=(abs(gen_diff) / max_size) * 200,  color=colors, edgecolor="white")
    circle1 = Line2D([], [], color="white", marker='o', markerfacecolor="green", markeredgecolor="white", markersize=10)
    circle2 = Line2D([], [], color="white", marker='o', markerfacecolor="red", markeredgecolor="white", markersize=10)
    circle3 = Line2D([], [], color="white", marker='o', markerfacecolor="white", markeredgecolor="black", markersize=10)

    unit = "TWh" if max_size > 1e3 else "GWh"
    max_size = max_size / 1e3 if max_size > 1e3 else max_size
    ax.legend((circle1, circle2, circle3), ('Increased generation in STST', 'Increased generation in EXP', f"max circle size: {round(max_size)} {unit}"), numpoints=1, loc="upper left")


    # always select same section
    xmin, ymin, xmax, ymax = df_stst_off.to_crs(crs.proj4_init).total_bounds
    pad = 1 * 1e5  # add a padding around the geometry
    ax.set_xlim(xmin-pad, xmax+pad)
    ax.set_ylim(ymin-pad, ymax+pad)

    ax.set_title(f"{carrier_renaming[carriers[i]] if carriers[i] in carrier_renaming else carriers[i]} capacity (STST - EXP)", fontsize=18, **font1)

# fig.suptitle("Spatial Differences in the capacity and generation for electricity generating VRE technologies (STST -EXP)", fontsize=16, **font1)
fig.tight_layout(pad=1)

#plt.close()
plt.show()

fig.savefig(f"{PLOT_DIR}01_general/4.2_systems_technologies/el_gen_vre_cap_gen_STST-EXP_map_all-ror.png")

In [ ]:
# cap diff in GW
df0 = pd.concat([df_stst_ons[f"{c}_cap"] - df_exp_ons[f"{c}_cap"], df_stst_ons[f"{c}_cap"], df_exp_ons[[f"{c}_cap"]]], axis=1)
df0.columns = ["1", "2", "3"]
(df0.sort_values(by="1", ascending=False)).dropna().head(10)

In [ ]:
# gen diff in TWh
df1 = pd.concat([df_stst_ons[f"{c}_gen"] - df_exp_ons[f"{c}_gen"], df_stst_ons[f"{c}_gen"], df_exp_ons[[f"{c}_gen"]]], axis=1)
df1.columns = ["1", "2", "3"]
(df1.sort_values(by="1", ascending=False) / 1000).dropna().tail(10)

In [ ]:
(df1.sort_values(by="1", ascending=False) / 1000).tail(10)

In [ ]:
df_stst_ons[f"{c}_cap"].describe()

In [ ]:
df_exp_ons[f"{c}_cap"].describe()

In [ ]:
# Storage capacity and generation difference
# ["battery discharger", "hydro", "V2G", "PHS"]

carriers = ["battery discharger", "V2G", "PHS"]

# STST - EXP
for c in carriers:
    # if one mv is nan the result is nan as well
    df_stst_ons[f"{c}_cap_STST-EXP"] = df_stst_ons[f"{c}_cap"] - df_exp_ons[f"{c}_cap"]
    
df = df_stst_ons
df_2 = df_exp_ons

fig, axs = plt.subplots(ncols=3, nrows=1, subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(15, 6), width_ratios=[1.25,1, 1])
crs = ccrs.EqualEarth()

plt.rcParams["figure.autolayout"] = True

for i, ax in enumerate(axs.reshape(-1)):
    
    if carriers[i] == "hydro":
        ax.axis('off')
        continue

    abs_max = max(abs(df[f"{carriers[i]}_cap_STST-EXP"].max()) , abs(df[f"{carriers[i]}_cap_STST-EXP"].min()))
    #unit_cap = "TW" if abs_max > 1e4 else "GW"
    #df[f"{carriers[i]}_cap_STST-EXP"] = (df[f"{carriers[i]}_cap_STST-EXP"] / 1e3) if abs_max > 1e4 else df[f"{carriers[i]}_cap_STST-EXP"]
    #abs_max = abs_max / 1e3 if abs_max > 1e4 else abs_max
    
    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')
    
    if carriers[i] == "battery discharger":

        df.to_crs(crs.proj4_init).plot(column=f"{carriers[i]}_cap_STST-EXP",
                                       ax=ax,
                                       cmap=plt.get_cmap('RdYlGn'),
                                       vmax=abs_max,
                                       vmin=-abs_max,
                                       linewidth=0.05,
                                       edgecolor = 'grey',
                                       legend=True,
                                       legend_kwds={'label':f"Capacity differences (GW)",'orientation': "vertical",'shrink' : 0.8}
                                       )


    # difference in generation (STST-EXP)
    gen_diff = df[f"{carriers[i]}_gen"] - df_2[f"{carriers[i]}_gen"]
    # red if negative and green if positive
    colors = ['red' if (x < 0) else 'green' for x in gen_diff ]

    max_size = abs(gen_diff).max()
    df.to_crs(crs.proj4_init).centroid.plot(ax=ax, sizes=(abs(gen_diff) / max_size) * 300,  color=colors, edgecolor="white")
    circle1 = Line2D([], [], color="white", marker='o', markerfacecolor="green", markeredgecolor="white", markersize=10)
    circle2 = Line2D([], [], color="white", marker='o', markerfacecolor="red", markeredgecolor="white", markersize=10)
    circle3 = Line2D([], [], color="white", marker='o', markerfacecolor="white", markeredgecolor="black", markersize=10)

    unit = "TWh" if max_size > 1e3 else "GWh"
    max_size = max_size / 1e3 if max_size > 1e3 else max_size
    ax.legend((circle1, circle2, circle3), ('Increased generation in STST', 'Increased generation in EXP', f"max circle size: {round(max_size)} {unit}"), numpoints=1, loc="upper left")


    # always select same section
    xmin, ymin, xmax, ymax = df_stst_ons.to_crs(crs.proj4_init).total_bounds
    pad = 1 * 1e5  # add a padding around the geometry
    ax.set_xlim(xmin-pad, xmax+pad)
    ax.set_ylim(ymin-pad, ymax+pad)
    

    ax.set_title(f"{carrier_renaming[carriers[i]] if carriers[i] in carrier_renaming else carriers[i]} capacity (STST - EXP)", fontsize=16, **font1)

# fig.suptitle("Spatial Differences in the capacity and generation for electricity generating Storage technologies (STST -EXP)", fontsize=16, **font1)
fig.tight_layout()

#plt.close()
plt.show()

fig.savefig(f"{PLOT_DIR}01_general/4.2_systems_technologies/el_gen_store_cap_gen_STST-EXP_map_all-hydro.png")

In [ ]:
c = "battery discharger"

# cap diff in GW
df0 = pd.concat([df_stst_ons[f"{c}_cap"] - df_exp_ons[f"{c}_cap"], df_stst_ons[f"{c}_cap"], df_exp_ons[[f"{c}_cap"]]], axis=1)
df0.columns = ["1", "2", "3"]
(df0.sort_values(by="1", ascending=False)).dropna().tail(10)

In [ ]:
# Peak plants capacity and generation difference
# ["urban central gas CHP", "urban central solid biomass CHP CC" ]

carriers = ["urban central gas CHP", "urban central solid biomass CHP CC" ]

# STST - EXP
for c in carriers:
    # if one mv is nan the result is nan as well
    df_stst_ons[f"{c}_cap_STST-EXP"] = df_stst_ons[f"{c}_cap"] - df_exp_ons[f"{c}_cap"]
    
df = df_stst_ons
df_2 = df_exp_ons

fig, axs = plt.subplots(ncols=2, nrows=int(len(carriers)/2), subplot_kw={'projection': ccrs.EqualEarth()},
                        figsize=(15, len(carriers)/2*6))
crs = ccrs.EqualEarth()

for i, ax in enumerate(axs.reshape(-1)):

    abs_max = max(abs(df[f"{carriers[i]}_cap_STST-EXP"].max()) , abs(df[f"{carriers[i]}_cap_STST-EXP"].min()))
    #unit_cap = "TW" if abs_max > 1e4 else "GW"
    #df[f"{carriers[i]}_cap_STST-EXP"] = (df[f"{carriers[i]}_cap_STST-EXP"] / 1e3) if abs_max > 1e4 else df[f"{carriers[i]}_cap_STST-EXP"]
    #abs_max = abs_max / 1e3 if abs_max > 1e4 else abs_max
    
    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.coastlines(edgecolor='black', linewidth=0.5)
    ax.set_facecolor('white')
    ax.add_feature(cartopy.feature.OCEAN, color='azure')

    df.to_crs(crs.proj4_init).plot(column=f"{carriers[i]}_cap_STST-EXP",
                                   ax=ax,
                                   cmap=plt.get_cmap('RdYlGn'),
                                   vmax=abs_max,
                                   vmin=-abs_max,
                                   linewidth=0.05,
                                   edgecolor = 'grey',
                                   legend=True,
                                   legend_kwds={'label':f"Capacity differences (GW)",'orientation': "vertical",'shrink' : 0.9}
                                   )

    # difference in generation (STST-EXP)
    gen_diff = df[f"{carriers[i]}_gen"] - df_2[f"{carriers[i]}_gen"]
    # red if negative and green if positive
    colors = ['red' if (x < 0) else 'green' for x in gen_diff ]

    max_size = abs(gen_diff).max()
    df.to_crs(crs.proj4_init).centroid.plot(ax=ax, sizes=(abs(gen_diff) / max_size) * 300,  color=colors, edgecolor="white")
    circle1 = Line2D([], [], color="white", marker='o', markerfacecolor="green", markeredgecolor="white", markersize=10)
    circle2 = Line2D([], [], color="white", marker='o', markerfacecolor="red", markeredgecolor="white", markersize=10)
    circle3 = Line2D([], [], color="white", marker='o', markerfacecolor="white", markeredgecolor="black", markersize=10)

    unit = "TWh" if max_size > 1e3 else "GWh"
    max_size = max_size / 1e3 if max_size > 1e3 else max_size
    ax.legend((circle1, circle2, circle3), ('Increased generation in STST', 'Increased generation in EXP', f"max circle size: {round(max_size)} {unit}"), numpoints=1, loc="upper left")


    # always select same section
    xmin, ymin, xmax, ymax = df_stst_ons.to_crs(crs.proj4_init).total_bounds
    pad = 1 * 1e5  # add a padding around the geometry
    ax.set_xlim(xmin-pad, xmax+pad)
    ax.set_ylim(ymin-pad, ymax+pad)

    ax.set_title(f"{carrier_renaming[carriers[i]] if carriers[i] in carrier_renaming else carriers[i]} capacity (STST - EXP)", fontsize=18, **font1)

# fig.suptitle("Spatial Differences in the capacity and generation for electricity generating peak plant technologies (STST -EXP)", fontsize=16, **font1)
fig.tight_layout()

#plt.close()
plt.show()

fig.savefig(f"{PLOT_DIR}01_general/4.2_systems_technologies/el_gen_peak_cap_gen_STST-EXP_map_all.png")

In [ ]:
c ="urban central solid biomass CHP CC" # "urban central gas CHP" # 
# cap diff in GW
df0 = pd.concat([df_stst_ons[f"{c}_cap"] - df_exp_ons[f"{c}_cap"], df_stst_ons[f"{c}_cap"], df_exp_ons[[f"{c}_cap"]]], axis=1)
df0.columns = ["1", "2", "3"]
(df0.sort_values(by="1", ascending=False)).dropna().tail(10)

### Full load hours

In [ ]:
# how are the full load hours aggregated:
# sum over the flh in every region adn time step (sumover time) -> you recieve flh for every region over the time

th = 0.9
n = n_no
df = df_no_ons

# get full_load hours (generators)
for carrier in n.generators.carrier.unique():
    gen = n.generators_t.p[n.generators[n.generators.carrier == carrier].index]
    # max_output = n.generators.p_nom_opt[n.generators.carrier == carrier]
    max_output = gen.max()

    # ts with max_output
    max_output_ts = gen.copy()
    for snap in n.storage_units_t.p.index:
        max_output_ts.loc[snap] = max_output[max_output_ts.columns]

    # calc full load hours
    flh = (gen >= th * max_output_ts).sum()
    flh.index = flh.index.map(n.generators.bus).map(n.buses.location)
    df[f"{carrier}_flh"] = flh

# links
for carrier in n.links.carrier.unique():
    gen = abs(n.links_t.p1.loc[:, n.links.carrier == carrier])
    # use p_nom_opt as indicator for full load hour
    # max_output = n.links.p_nom_opt[n.links.carrier == carrier]
    # use maximum of generation output as indicator for full load hour
    max_output = gen.max()

    # ts with p_nom_opt
    max_output_ts = gen.copy()
    for snap in n.storage_units_t.p.index:
        max_output_ts.loc[snap] = max_output[max_output_ts.columns]

    # calc full load hours
    flh = (gen >= th * max_output_ts).sum()
    flh.index = flh.index.map(n.links.bus1).map(n.buses.location)
    # group duplicate index entries
    flh = flh.groupby(by=["Link"], axis="index").sum()
    df[f"{carrier}_flh"] = flh

# storage units
for carrier in n.storage_units.carrier.unique():
    gen = n.storage_units_t.p_dispatch.loc[:, n.storage_units.carrier == carrier]
    max_output = gen.max()

    max_output_ts = gen.copy()
    for snap in n.storage_units_t.p.index:
        max_output_ts.loc[snap] = max_output[max_output_ts.columns]

    # calc full load hours
    flh = (gen >= th * max_output_ts).sum()
    flh.index = flh.index.map(n.storage_units.bus).map(n.buses.location)
    df[f"{carrier}_flh"] = flh

df.head(3)

In [ ]:
# full load hours
boxprops=dict(facecolor="white", color="white")
medianprops = dict(color="black",linewidth=1.5)
meanprops = {"marker":"d","markerfacecolor":"white", "markeredgecolor":"black"}

i_no_gen_rev = [carrier_renaming_reverse.get(n, n) for n in i_no_gen]
# multiply with 3 to account for the 3h resolution
df = df_no_ons[[f"{i}_flh" for i in i_no_gen_rev]]*3
df = df.values

# Filter data using np.isnan
mask = ~np.isnan(df)
filtered_data = [d[m] for d, m in zip(df.T, mask.T)]

fig, ax = plt.subplots(figsize=(12, 4))
bp = plt.boxplot(x=filtered_data, labels =i_no_gen, patch_artist=True, showmeans=True, boxprops=boxprops, meanprops=meanprops, medianprops=medianprops)

for box, col in zip(bp['boxes'],[carrier_colors[c] for c in i_no_gen]):
    # change outline color
    box.set(color=col, linewidth=2)

plt.title("Title")
ax.patch.set_facecolor('lightgrey')
ax.patch.set_alpha(0.5)
plt.xticks(rotation=90)
plt.title("Full load hours", fontsize=16, pad=15,  **font1)
plt.show()

### Utilitisation rate
(calc shifted to 00_CALC notebook)

In [ ]:
# utilisation rates across regions as boxplot (STST & EXP)

# boxplot propertes
medianprops = dict(color="black",linewidth=1.5)
meanprops = {"marker":"d","markerfacecolor":"white", "markeredgecolor":"black"}
flierprops= {'marker': 'x', 'markersize': 5, 'markeredgecolor': 'black'}
whiskerprops = dict(linestyle='-',linewidth=1.0, color='black')

# data
stst_el_gen = df_stst_ons[[c + "_ur" for c in c_el_gen_s]]
stst_el_gen = stst_el_gen.values
exp_el_gen = df_exp_ons[[c + "_ur" for c in c_el_gen_s]]
exp_el_gen = exp_el_gen.values

# Filter data using np.isnan
mask_stst = ~np.isnan(stst_el_gen)
filtered_stst = [d[m] for d, m in zip(stst_el_gen.T, mask_stst.T)]
mask_exp = ~np.isnan(exp_el_gen)
filtered_exp = [d[m] for d, m in zip(exp_el_gen.T, mask_exp.T)]

ticks = [carrier_renaming.get(n, n) for n in c_el_gen_s]
index = c_el_gen_s
fig, ax = plt.subplots(figsize=(12, 6))

stst_plot = plt.boxplot(filtered_stst,
                        positions=np.array(np.arange(len(ticks)))*2.0-0.35,
                        widths=0.6,
                        patch_artist=True,
                        showmeans=False,
                        meanprops=meanprops,
                        medianprops=medianprops,
                        flierprops=flierprops,
                        whiskerprops=whiskerprops,
                        zorder=1
                        )

exp_plot = plt.boxplot(filtered_exp,
                       positions=np.array(np.arange(len(ticks)))*2.0+0.35,
                       widths=0.6,
                       patch_artist=True,
                       showmeans=False,
                       meanprops=meanprops,
                       medianprops=medianprops,
                       flierprops=flierprops,
                       whiskerprops=whiskerprops,
                       zorder=2
                       )


for box, col in zip(stst_plot['boxes'],[carrier_colors[c] for c in index]):
    # change outline color
    box.set_facecolor(col)
    box.set_linestyle('--')

for box, col in zip(exp_plot['boxes'],[carrier_colors[c] for c in index]):
    # change outline color
    box.set_facecolor(col)

# sample sizes
#for i, sample_size in enumerate(df_stst_ons[[f"{i}_ur" for i in index]].count()):
#    ax.annotate(sample_size, xy=(0,0),  xycoords='axes fraction',
#        xytext=((i+0.3)/len(index),1.01), textcoords='axes fraction', color="blue")

#for i, sample_size in enumerate(df_exp_ons[[f"{i}_ur" for i in index]].count()):
#    ax.annotate(sample_size, xy=(0,0),  xycoords='axes fraction',
#        xytext=((i+0.6)/len(index),1.01), textcoords='axes fraction', color="red")
    
# annotate both sample sizes
sample_size1 = df_stst_ons[[f"{i}_ur" for i in index]].count()
sample_size2 = df_exp_ons[[f"{i}_ur" for i in index]].count()

for i, sz1, sz2 in zip(np.arange(len(index)), sample_size1, sample_size2):
    ax.annotate(f"({sz1} / {sz2} )", xy=(0,0),  xycoords='axes fraction',
        xytext=((i+0.15)/len(index),1.01), textcoords='axes fraction', color="blue")

# overall ur
ax.plot(np.array(np.arange(len(ticks)))*2.0-0.35, overall_ur_stst[index].transpose(),"x", marker='*', color="red", markersize= 10, markerfacecolor="white", zorder=3)
ax.plot(np.array(np.arange(len(ticks)))*2.0+0.35, overall_ur_exp[index].transpose(),"x", marker='*', color="red", markersize= 10, markerfacecolor="white",zorder=4)
 
# explanations
plt.xticks(np.arange(0, len(ticks) * 2, 2), ticks)
plt.xticks(rotation=90)
# plt.title("Market values of electricity generating technologies across the regions (STST vs. EXP)", fontsize=16, pad=20,  **font1)

# cosmetics
ax.patch.set_facecolor('lightgrey')
ax.patch.set_alpha(0.5)
ax.grid(True)
ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
ax.set_ylabel("utilisation rate [%]")

# legend
patch1 = matplotlib.patches.Patch(ls="--", facecolor="white", edgecolor="black")
patch2 = matplotlib.patches.Patch(ls="-", facecolor="white", edgecolor="black")
ax.legend([patch1, patch2], ['STST', 'EXP'], loc="lower left")

fig.tight_layout()
plt.show()

fig.savefig(f"{PLOT_DIR}01_general/4.2_systems_technologies/el_gen_ur_STST_EXP.png")

In [ ]:
overall_ur_stst[c_el_gen_s]

In [ ]:
overall_ur_exp[c_el_gen_s]

In [ ]:
(overall_ur_exp - overall_ur_stst) [c_el_gen_s]

In [ ]:
# 1: what does it mean if the star is outside the box / whiskers: star below box (hydro) means that most of the regions are having higher ur than the overall utilitization. The regions with most generation have a lower ur than the average

# 2: what is the difference between the capacity facotr and the utilitization rate for technologies with a constant capacity e.g. gas CHP? Shouldnt this be the same? Because it is not.

### Generation profiles

In [ ]:
# generators, links, and storage units
carriers = ["battery discharger", "hydro"]
n = n_no

fig, axs = plt.subplots(nrows=1,ncols=2, figsize=(18, 6))

for i, ax in enumerate(axs):

    if carriers[i] in n.generators.carrier.unique().tolist():
        # get generation in TWh
        df = pd.DataFrame(n.generators_t.p.loc[:, n.generators.carrier == carriers[i]].sum(axis=1)) / 1000

    elif carriers[i] in n.links.carrier.unique().tolist():
        # get generation in TWh
        df = pd.DataFrame(n.links_t.p0.loc[:, n.links.carrier == carriers[i]].sum(axis=1)) / 1000

    elif carriers[i] in n.storage_units.carrier.unique().tolist():
        # get generation in TWh
        df = pd.DataFrame(n.storage_units_t.p_dispatch.loc[:, n.storage_units.carrier == carriers[i]].sum(axis=1)) / 1000

    hours = df.index.hour.unique()[::-1]
    df_start = pd.DataFrame(index=pd.Index(df.index.date).unique())

    for hour in hours:
        df_start[str(hour)] = df[df.index.hour==hour].values

    sns.heatmap(df_start.transpose(),
                ax=ax,
                cmap=plt.get_cmap("magma_r"),
                linewidth=0.001,
                xticklabels=15,
                cbar_kws={'label': 'Generation in TWh', 'pad': 0.09}
                )


    ax.set_title(f"{carriers[i]}", fontsize=16, **font1)
    ax.set_ylabel("hour of the day", fontsize=12, **font1)
    ax.set_xlabel("day of the year", fontsize=12, **font1)
    #plt.colorbar(im,fraction=0.046, pad=0.04)

    # Rewrite the y labels
    ax_labels = ax.get_xticks()
    ax.xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%d-%b'))

    # Plot generation profile on second y-axis
    # resample and transform axis of date to index (heatmap works not with dates)
    df2 = df.resample("5D").sum()
    df2 = df2[df2.index.year == 2013]
    df2.index = df2.index.dayofyear - 0.5

    ax2 = ax.twinx()
    ax2.plot(df2, color="white", lw=2, path_effects=[pe.Stroke(linewidth=3, foreground='black'), pe.Normal()])
    ax2.legend( loc='lower right', labels=['Daily generation (right axis)'])
    ax2.set_ylabel("Sum of daily generation in TWh", fontsize=12, **font1)
    ax2.grid(False)

fig.tight_layout(pad=3)
plt.show()

In [ ]:
# thesis plot
# Generation heatmap and average: VRE
["onwind", "ror", "solar", "solar rooftop", "offwind-dc", "offwind-ac"]

carriers = ["onwind", "solar", "offwind-dc", "ror"]
model = "STST"

if model == "STST":
    n = n_no

elif model == "EXP":
    n = n_h2

fig, axs = plt.subplots(nrows=int(len(carriers)/2), ncols=2, figsize=(16, len(carriers)/2*4.5))

for i, ax in enumerate(axs.reshape(-1)):

    if carriers[i] in n.generators.carrier.unique().tolist():
        # get generation in TWh
        df = pd.DataFrame(n.generators_t.p.loc[:, n.generators.carrier == carriers[i]].sum(axis=1)) / 1000 * 3

    elif carriers[i] in n.links.carrier.unique().tolist():
        # get generation in TWh
        df = pd.DataFrame(n.links_t.p0.loc[:, n.links.carrier == carriers[i]].sum(axis=1)) / 1000 * 3

    elif carriers[i] in n.storage_units.carrier.unique().tolist():
        # get generation in TWh
        df = pd.DataFrame(n.storage_units_t.p_dispatch.loc[:, n.storage_units.carrier == carriers[i]].sum(axis=1)) / 1000 * 3

    else:
        print(f"{carriers[i]} does not exist!")


    hours = df.index.hour.unique()[::-1]
    df_start = pd.DataFrame(index=pd.Index(df.index.date).unique())

    for hour in hours:
        df_start[str(hour)] = df[df.index.hour==hour].values

    sns.heatmap(df_start.transpose(),
                ax=ax,
                cmap=plt.get_cmap("magma_r"),
                linewidth=0.001,
                xticklabels=15,
                cbar_kws={'label': 'Generation in GWh', 'pad': 0.1})

    ax.set_title(f"{carriers[i]}", fontsize=18, **font1)
    ax.set_ylabel("hour of the day", fontsize=12, **font1)
    ax.set_xlabel("day of the year", fontsize=12, **font1)

    # Rewrite the y labels
    x_labels = ax.get_xticks()
    ax.xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%d-%b'))

    # Plot generation profile on second y-axis
    # resample and transform axis of date to index (heatmap works not with dates)
    df2 = df.resample("5D").sum() / 1000
    df2 = df2[df2.index.year == 2013]
    df2.index = df2.index.dayofyear - 0.5

    ax2 = ax.twinx()
    ax2.plot(df2, color="white", lw=2, path_effects=[pe.Stroke(linewidth=3, foreground='black'), pe.Normal()])
    ax2.legend( loc='lower right', labels=['Generation (right axis)'])
    ax2.set_ylabel("Generation in TWh (5-day sum)", fontsize=12, **font1)
    ax2.grid(False)

fig.tight_layout(pad=3)
plt.show()

fig.savefig(f"{PLOT_DIR}01_general/4.2_systems_technologies/el_gen_profile_vre_{model}.png")

In [ ]:
# thesis plot
# Generation heatmap and average: Storage

carriers = ["battery discharger", "hydro", "V2G", "PHS"]
model = "EXP"

if model == "STST":
    n = n_no

elif model == "EXP":
    n = n_h2


fig, axs = plt.subplots(nrows=2,ncols=2, figsize=(16, 9))

for i, ax in enumerate(axs.reshape(-1)):

    if carriers[i] in n.generators.carrier.unique().tolist():
        # get generation in TWh
        df = pd.DataFrame(n.generators_t.p.loc[:, n.generators.carrier == carriers[i]].sum(axis=1)) / 1000 * 3

    elif carriers[i] in n.links.carrier.unique().tolist():
        # get generation in TWh
        df = pd.DataFrame(n.links_t.p0.loc[:, n.links.carrier == carriers[i]].sum(axis=1)) / 1000 * 3

    elif carriers[i] in n.storage_units.carrier.unique().tolist():
        # get generation in TWh
        df = pd.DataFrame(n.storage_units_t.p_dispatch.loc[:, n.storage_units.carrier == carriers[i]].sum(axis=1)) / 1000 * 3

    else:
        print(f"{carriers[i]} does not exist!")


    hours = df.index.hour.unique()[::-1]
    df_start = pd.DataFrame(index=pd.Index(df.index.date).unique())

    for hour in hours:
        df_start[str(hour)] = df[df.index.hour==hour].values

    sns.heatmap(df_start.transpose(),
                ax=ax,
                cmap=plt.get_cmap("magma_r"),
                linewidth=0.001,
                xticklabels=15,
                cbar_kws={'label': 'Generation in GWh', 'pad': 0.1})

    ax.set_title(f"{carriers[i]}", fontsize=18, **font1)
    ax.set_ylabel("hour of the day", fontsize=12, **font1)
    ax.set_xlabel("day of the year", fontsize=12, **font1)

    # Rewrite the y labels
    x_labels = ax.get_xticks()
    ax.xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%d-%b'))

    # Plot generation profile on second y-axis
    # resample and transform axis of date to index (heatmap works not with dates)
    df2 = df.resample("5D").sum() / 1000
    df2 = df2[df2.index.year == 2013]
    df2.index = df2.index.dayofyear - 0.5

    ax2 = ax.twinx()
    ax2.plot(df2, color="white", lw=2, path_effects=[pe.Stroke(linewidth=3, foreground='black'), pe.Normal()])
    ax2.legend( loc='lower right', labels=['Generation (right axis)'])
    ax2.set_ylabel("Generation in TWh (5-day sum)", fontsize=12, **font1)
    ax2.grid(False)

fig.tight_layout(pad=3)
plt.show()

fig.savefig(f"{PLOT_DIR}01_general/4.2_systems_technologies/el_gen_profile_store_{model}.png")

In [ ]:
# thesis plot
# Generation heatmap and average: Peakplants

carriers = ["urban central gas CHP", "urban central solid biomass CHP CC" ]
model = "STST"

if model == "STST":
    n = n_no

elif model == "EXP":
    n = n_h2


fig, axs = plt.subplots(nrows=1,ncols=2, figsize=(16, 4.5))

for i, ax in enumerate(axs):

    if carriers[i] in n.generators.carrier.unique().tolist():
        # get generation in TWh
        df = pd.DataFrame(n.generators_t.p.loc[:, n.generators.carrier == carriers[i]].sum(axis=1)) / 1000 * 3

    elif carriers[i] in n.links.carrier.unique().tolist():
        # get generation in TWh
        df = pd.DataFrame(n.links_t.p0.loc[:, n.links.carrier == carriers[i]].sum(axis=1)) / 1000 * 3

    elif carriers[i] in n.storage_units.carrier.unique().tolist():
        # get generation in TWh
        df = pd.DataFrame(n.storage_units_t.p_dispatch.loc[:, n.storage_units.carrier == carriers[i]].sum(axis=1)) / 1000 * 3

    else:
        print(f"{carriers[i]} does not exist!")


    hours = df.index.hour.unique()[::-1]
    df_start = pd.DataFrame(index=pd.Index(df.index.date).unique())

    for hour in hours:
        df_start[str(hour)] = df[df.index.hour==hour].values

    sns.heatmap(df_start.transpose(),
                ax=ax,
                cmap=plt.get_cmap("magma_r"),
                linewidth=0.001,
                xticklabels=15,
                cbar_kws={'label': 'Generation in GWh', 'pad': 0.1})

    ax.set_title(carriers[i].replace("urban central solid ", "").replace("urban central ", ""), fontsize=18, **font1)
    ax.set_ylabel("hour of the day", fontsize=12, **font1)
    ax.set_xlabel("day of the year", fontsize=12, **font1)

    # Rewrite the y labels
    x_labels = ax.get_xticks()
    ax.xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%d-%b'))

    # Plot generation profile on second y-axis
    # resample and transform axis of date to index (heatmap works not with dates)
    df2 = df.resample("5D").sum() / 1000
    df2 = df2[df2.index.year == 2013]
    df2.index = df2.index.dayofyear - 0.5

    ax2 = ax.twinx()
    ax2.plot(df2, color="white", lw=2, path_effects=[pe.Stroke(linewidth=3, foreground='black'), pe.Normal()])
    ax2.legend( loc='lower right', labels=['Generation (right axis)'])
    ax2.set_ylabel("Generation in TWh (5-day sum)", fontsize=12, **font1)
    ax2.grid(False)


fig.tight_layout(pad=3)
plt.show()

fig.savefig(f"{PLOT_DIR}01_general/4.2_systems_technologies/el_gen_profile_peak_{model}.png")

### Storage

In [ ]:
### hydro generation
# plt.plot(n.storage_units_t.p_dispatch.loc[:, n.storage_units.carrier == "hydro"].sum(axis=1))

In [ ]:
### hydro water availability
plt.plot(n.storage_units_t.inflow.loc[:, n.storage_units.carrier == "hydro"].sum(axis=1).cumsum())

In [ ]:
### hydro water availability
plt.plot(n.storage_units_t.spill.loc[:, n.storage_units.carrier == "hydro"].sum(axis=1))

In [ ]:
# State of charge [per unit of max] (all stores and storage units)
# Ratio of total generation of max state of charge

gen_charge_ratio= []
max_charge= []

st_carriers = ["battery", "Li ion", "hydro", "PHS", "H2"] # "battery", "Li ion",
carriers = ["battery discharger", "V2G", "hydro", "PHS", "H2"] # "battery discharger", "V2G",
period= "2013"
model = "STST"

if model == "STST":
    n = n_no

elif model == "EXP":
    n = n_h2


stor_res = pd.DataFrame(index = st_carriers, columns =["max_charge", "gen_charge_ratio", "max_stor_cap", "gen_charge_cap_ratio", "gen_sum" ])

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(9,10), height_ratios=[1, 1.5])

for i, c in enumerate(st_carriers):
    if c in n.storage_units.carrier.unique().tolist():
        # state of charge (sum over locations: 2920 values)
        charge = n.storage_units_t.state_of_charge.loc[:, n.storage_units.carrier == c].sum(axis=1)[period]
        gen_sum = n.storage_units_t.p_dispatch.loc[period, n.storage_units.carrier == carriers[i]].sum().sum()
        gen = n.storage_units_t.p_dispatch.loc[period, n.storage_units.carrier == carriers[i]].sum(axis=1)
        index = n.storage_units[n.storage_units.carrier == c].index
        max_stor_cap = (n.storage_units.max_hours * n.storage_units.p_nom_opt)[index].sum()
        stor_res.loc[c, "max_charge"] = charge.max()
        stor_res.loc[c, "gen_charge_ratio"] = gen_sum/charge.max()
        stor_res.loc[c, "max_stor_cap"] = max_stor_cap
        stor_res.loc[c, "gen_charge_cap_ratio"] = gen_sum/max_stor_cap
        stor_res.loc[c, "gen_sum"] = gen_sum

    elif c in n.stores.carrier.unique().tolist():
        # state of charge (sum over locations: 2920 values)
        charge = n.stores_t.e.loc[:, n.stores.carrier == c].sum(axis=1)[period]
        gen_sum = -n.links_t.p1.loc[period, n.links.carrier == carriers[i]].sum().sum()
        max_stor_cap = n.stores.e_nom_opt[n.stores[n.stores.carrier == c].index].sum()
        stor_res.loc[c, "max_charge"] = charge.max()
        stor_res.loc[c, "gen_charge_ratio"] = gen_sum/charge.max()
        stor_res.loc[c, "max_stor_cap"] = max_stor_cap
        stor_res.loc[c, "gen_charge_cap_ratio"] = gen_sum/max_stor_cap
        stor_res.loc[c, "gen_sum"] = gen_sum
        
    if c in ["battery", "Li ion"]:
        ax1.plot(charge/max_stor_cap, label=c, color=carrier_colors[c], marker=markers[i], markevery=[0], mfc='white', mec='black')
        
    else:
        ax2.plot(charge/max_stor_cap, label=c, color=carrier_colors[c], marker=markers[i], markevery=[0], mfc='white', mec='black')

ax1.set_title("State of charge of short-term storage technologies")
ax2.set_title("State of charge of mid- and long-term storage technologies")
ax1.set_ylabel("State of charge [per unit of max storage capacity]")
ax2.set_ylabel("State of charge [per unit of max storage capacity]")
ax1.legend(loc="lower right")
ax2.legend(loc="lower right")
               
fig.tight_layout(pad=3)
plt.show()

fig.savefig(f"{PLOT_DIR}01_general/4.2_systems_technologies/charge_state_store_{model}.png")

In [ ]:
# Ratio of total generation of max state of charge
stor_res["max_charge_max_stor_cap_ratio"] =  stor_res["max_charge"] / stor_res["max_stor_cap"]
stor_res

In [ ]:
# TWh
stor_res["max_stor_cap"] / 1e6

In [ ]:
# how do I get the maximum storage capacity?

# stores:
c = "battery"
index = n.stores[n.stores.carrier == c].index
n.stores.e_nom_opt[index]

# storage units:
c = "hydro"
index = n.storage_units[n.storage_units.carrier == c].index
(n.storage_units.max_hours * n.storage_units.p_nom_opt)[index].sum()

In [ ]:
# State of charge [per unit of max] (all stores and storage units)
# Ratio of total generation of max state of charge

gen_charge_ratio= []
max_charge= []

carriers = ["battery", "battery discharger", "battery charger"]
n = n_no
start_date = "2013-06-1 00:00:00"
end_date = "2013-06-09 00:00:00"
period = n_no.generators_t.p.index[(n_no.generators_t.p.index >= start_date) & (n_no.generators_t.p.index <= end_date)]

fig = plt.figure(figsize=(8,4))

for i, c in enumerate(carriers):
    if c in n.storage_units.carrier.unique().tolist():
        # state of charge (sum over locations: 2920 values)
        charge = n.storage_units_t.state_of_charge.loc[:, n.storage_units.carrier == c].sum(axis=1)[period]
        gen_sum = n.storage_units_t.p_dispatch.loc[period, n.storage_units.carrier == carriers[i]].sum().sum()
        gen = n.storage_units_t.p_dispatch.loc[period, n.storage_units.carrier == carriers[i]].sum(axis=1)
        index = n.storage_units[n.storage_units.carrier == c].index
        max_stor_cap = (n.storage_units.max_hours * n.storage_units.p_nom_opt)[index].sum()
        stor_res.loc[c, "max_charge"] = charge.max()
        stor_res.loc[c, "gen_charge_ratio"] = gen_sum/charge.max()
        stor_res.loc[c, "max_stor_cap"] = max_stor_cap

        plt.plot(charge/max_stor_cap, label=c, color=carrier_colors[c], marker=markers[i], markevery=[0], mfc='white', mec='black',linewidth=3)
        plt.plot(gen/gen.max(), label=f"{c}", color=carrier_colors[c], marker=markers[i], markevery=[0], mfc='white', mec='black', ls="--", linewidth=3)

    elif c in n.links.carrier.unique().tolist():
        gen = -n.links_t.p1.loc[period, n.links.carrier == c].sum(axis=1)

        plt.plot(gen/gen.max(), label=f"{c}", color=carrier_colors[c], marker=markers[i], markevery=[0], mfc='white', mec='black', ls=":", linewidth=3)

    elif c in n.stores.carrier.unique().tolist():
        # state of charge (sum over locations: 2920 values)
        charge = n.stores_t.e.loc[:, n.stores.carrier == c].sum(axis=1)[period]
        gen_sum = -n.links_t.p1.loc[period, n.links.carrier == carriers[i]].sum().sum()
        max_stor_cap = n.stores.e_nom_opt[n.stores[n.stores.carrier == c].index].sum()
        stor_res.loc[c, "max_charge"] = charge.max()
        stor_res.loc[c, "gen_charge_ratio"] = gen_sum/charge.max()
        stor_res.loc[c, "max_stor_cap"] = max_stor_cap

        plt.plot(charge/max_stor_cap, label=c, color=carrier_colors[c], marker=markers[i], markevery=[0], mfc='white', mec='black',linewidth=3)

plt.gca().xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%H')) # %d-%m
plt.gca().xaxis.set_major_locator(mtick.MultipleLocator(0.125))
plt.ylabel("State of charge and generation [per unit of max]")
plt.xlabel("Hour of the day")
plt.legend(loc="upper right")
plt.tight_layout()
plt.show()


In [ ]:
# State of charge [per unit of max] (all stores and storage units)
# Ratio of total generation of max state of charge

temp_colors = {
    'PHS': 'darkorange',
    'hydro': 'navy'
}

carrier_lists =[["battery", "battery discharger", "battery charger"],["hydro", "PHS"]]
model = "EXP"

if model == "STST":
    n = n_no

elif model == "EXP":
    n = n_h2

start_date = "2013-06-6 00:00:00"
end_date = "2013-06-10 00:00:00"
period = n.generators_t.p.index[(n.generators_t.p.index >= start_date) & (n.generators_t.p.index <= end_date)]

fig, axs = plt.subplots(nrows=2,ncols=1, figsize=(10,6))

for j, ax in enumerate(axs):
    carriers = carrier_lists[j]

    for i, c in enumerate(carriers):
        if c in n.storage_units.carrier.unique().tolist():
            # state of charge (sum over locations: 2920 values)
            charge = n.storage_units_t.state_of_charge.loc[:, n.storage_units.carrier == c].sum(axis=1)[period]
            gen_sum = n.storage_units_t.p_dispatch.loc[period, n.storage_units.carrier == carriers[i]].sum().sum()
            gen = n.storage_units_t.p_dispatch.loc[period, n.storage_units.carrier == carriers[i]].sum(axis=1)
            index = n.storage_units[n.storage_units.carrier == c].index
            max_stor_cap = (n.storage_units.max_hours * n.storage_units.p_nom_opt)[index].sum()
            stor_res.loc[c, "max_charge"] = charge.max()
            stor_res.loc[c, "gen_charge_ratio"] = gen_sum/charge.max()
            stor_res.loc[c, "max_stor_cap"] = max_stor_cap

            ax.plot(charge/max_stor_cap, label=c, color=temp_colors[c], marker=markers[i], markevery=[0], mfc='white', mec='black',linewidth=2)
            ax.plot(gen/gen.max(), label=f"{c} (generation)", color=temp_colors[c], marker=markers[i], markevery=[0], mfc='white', mec='black', ls=":", linewidth=3)

        elif c in n.links.carrier.unique().tolist():
            gen = -n.links_t.p1.loc[period, n.links.carrier == c].sum(axis=1)

            ax.plot(gen/gen.max(), label=f"{c}", color=carrier_colors[c], marker=markers[i], markevery=[0], mfc='white', mec='black', ls=":", linewidth=3)

        elif c in n.stores.carrier.unique().tolist():
            # state of charge (sum over locations: 2920 values)
            charge = n.stores_t.e.loc[:, n.stores.carrier == c].sum(axis=1)[period]
            gen_sum = -n.links_t.p1.loc[period, n.links.carrier == carriers[i]].sum().sum()
            max_stor_cap = n.stores.e_nom_opt[n.stores[n.stores.carrier == c].index].sum()
            stor_res.loc[c, "max_charge"] = charge.max()
            stor_res.loc[c, "gen_charge_ratio"] = gen_sum/charge.max()
            stor_res.loc[c, "max_stor_cap"] = max_stor_cap

            ax.plot(charge/max_stor_cap, label=c, color=carrier_colors[c], marker=markers[i], markevery=[0], mfc='white', mec='black',linewidth=3)

    ax.xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%H')) # %d-%m
    ax.xaxis.set_major_locator(mtick.MultipleLocator(0.125))
    ax.set_ylabel("Charge / generation [per unit of max]")
    ax.set_xlabel("Hour of the day")
    ax.legend(loc="upper right")

plt.tight_layout()
plt.show()

# fig.savefig(f"{PLOT_DIR}01_general/4.2_systems_technologies/charge_state_store_daily_{model}.png")

In [ ]:
stor_res

In [ ]:
charge = n.stores_t.e.loc[:, n.stores.carrier == "battery"].sum(axis=1)
gen = n.links_t.p1.loc[:, n.links.carrier == "battery discharger"].sum().sum()

In [ ]:
# State of charge [per unit of max]
carrier = "H2"

# state of charge (sum over locations: 2920 values)
charge = n.stores_t.e.loc[:, n.stores.carrier == carrier].sum(axis=1)

In [ ]:
plt.plot(charge/charge.max())

#### How long is one unit on average stored?
- LIFO
- Distribution of storage time over the year
- watch out as the smallest can be ony minimum 3h steps

In [ ]:
temp_colors = {
    'PHS': 'purple',
    'hydro': 'navy',
    'battery': 'hotpink',
    'gas': 'orange',
    'H2': 'turquoise',
    'Li ion': 'greenyellow',
    'home battery': 'violet',
    'oil': 'k',
    'solid biomass': 'sandybrown',
    'biogas': 'palegoldenrod',
}

In [ ]:
# singel case
c = "battery"
charge = n.stores_t.e.loc[:, n.stores.carrier == c].sum(axis=1)
result = time_stored_LIFO(charge)

amounts = pd.DataFrame(result[[f"amount{i}" for i in range(1,int(((len(result.columns) - 3) / 2)))]].values.flatten())
timedeltas = pd.DataFrame(result[[f"timedelta{i}" for i in range(1,int(((len(result.columns) - 3) / 2)))]].values.flatten())

all_df = pd.concat([ amounts, timedeltas], axis=1).dropna()
all_df.columns = ["amounts", "timedeltas"]
all_df["hours"] = (all_df["timedeltas"] /  timedelta(hours=1)).astype(int)

In [ ]:
# Barplot: duration of storage per amountof stored energy

average_storing_time = pd.DataFrame(index=range(1))
carriers = ["battery", "PHS", "hydro", "gas", "H2", "Li ion", "oil"]
model = "EXP"

if model == "STST":
    n = n_no

elif model == "EXP":
    n = n_h2
        
start_date = "2013-06-6 00:00:00"
end_date = "2013-06-10 00:00:00"
period = n.generators_t.p.index[(n.generators_t.p.index >= start_date) & (n.generators_t.p.index <= end_date)]
period="2013"

bins = [0, 6, 12, 18, 24, 2*24, 3*24, 7*24, 14*24, 30*24,60*24,120*24, np.inf]
bins_labels = ["[0h,6h]", "(6h,12h]", "(12h,18h]", "(18h,24h]", "(24h,2d]","(2d,3d]","(3d,7d]", "(7d,14d]", "(14d,30d]","(30d,60d]","(60d,120d]", " > 120d" ]

res_grouped = pd.DataFrame(index= bins_labels)

for c in carriers:
        if c in n.storage_units.carrier.unique().tolist():
            # state of charge (sum over locations: 2920 values)
            charge = n.storage_units_t.state_of_charge.loc[:, n.storage_units.carrier == c].sum(axis=1)[period]

        elif c in n.stores.carrier.unique().tolist():
            # state of charge (sum over locations: 2920 values)
            charge = n.stores_t.e.loc[:, n.stores.carrier == c].sum(axis=1)[period]

        else:
            print(f"{c} does not exist!")

        result = time_stored_LIFO(charge)

        # get amounts and timedeltas
        i = int(((len(result.columns) - 3) / 2))
        amounts = pd.DataFrame(result[[f"amount{i}" for i in range(1,i+1)]].values.flatten(order='C'))
        timedeltas = pd.DataFrame(result[[f"timedelta{i}" for i in range(1,i+1)]].values.flatten(order='C'))
        all_df = pd.concat([ amounts, timedeltas], axis=1).dropna()
        all_df.columns = ["amounts", "timedeltas"]
        all_df["hours"] = (all_df["timedeltas"] /  timedelta(hours=1)).astype(int)
        
        # how long is one unit on average stored
        average_storing_time[c] = (all_df.amounts * all_df.hours / all_df.amounts.sum()).sum()

        # groupby hours and normalise
        all_df_grouped = all_df[["amounts", "hours"]].groupby(by="hours").sum() / all_df["amounts"].sum()

        #binning
        all_df_grouped["bins"] = pd.cut(x=all_df_grouped.index, bins=bins, labels=bins_labels, right=True)

        # save to result
        res_grouped[c] = all_df_grouped[["amounts", "bins"]].groupby(by="bins").sum()
        

fig = res_grouped.reset_index().plot(x="index", y=carriers, kind="bar",
                               rot=0,
                               figsize=(11,4),
                               color=[temp_colors[c] for c in carriers],
                               xlabel="",
                               ylabel="Amount of stored energy (%)",
                               width=0.85
                               ).get_figure()
plt.show()

fig.savefig(f"{PLOT_DIR}01_general/4.2_systems_technologies/storage_time_bars_{model}.png")

In [ ]:
# discharged after only 6 hours or less
res_grouped.loc[:"[0h,6h]"].sum()

In [ ]:
# discharged withing 24 hours
res_grouped.loc[:"(18h,24h]"].sum()

In [ ]:
# discharged within one week until 4 months
res_grouped.loc["(7d,14d]":"(60d,120d]"].sum()

In [ ]:
# discharged within 18 hours
res_grouped.loc[:"(12h,18h]"].sum()

In [ ]:
# discharged within one week until 4 months
res_grouped.loc["(24h,2d]":"(30d,60d]"].sum()

In [ ]:
res_grouped

In [ ]:
average_storing_time

In [ ]:
average_storing_time / 24

In [ ]:
# how long is one unit on average stored
(all_df.amounts * all_df.hours / all_df.amounts.sum()).sum()

In [ ]:
# Distribution of storage time over the year

carriers = ["battery", "PHS", "hydro", "gas", "H2", "Li ion", "biogas", "solid biomass", "oil", "home battery" ]
model = "EXP"

if model == "STST":
    n = n_no

elif model == "EXP":
    n = n_h2

start_date = "2013-06-6 00:00:00"
end_date = "2013-06-10 00:00:00"
period = n.generators_t.p.index[(n.generators_t.p.index >= start_date) & (n.generators_t.p.index <= end_date)]
period="2013"

res = pd.DataFrame(index= np.arange(1,13))

for c in carriers:
        if c in n.storage_units.carrier.unique().tolist():
            # state of charge (sum over locations: 2920 values)
            charge = n.storage_units_t.state_of_charge.loc[:, n.storage_units.carrier == c].sum(axis=1)[period]

        elif c in n.stores.carrier.unique().tolist():
            # state of charge (sum over locations: 2920 values)
            charge = n.stores_t.e.loc[:, n.stores.carrier == c].sum(axis=1)[period]

        else:
            print(f"{c} does not exist!")

        result = time_stored_LIFO(charge)

        # get amounts and timedeltas
        i = int(((len(result.columns) - 3) / 2))
        amounts = pd.DataFrame(result[[f"amount{i}" for i in range(1,i+1)]].values.flatten(order='C'))
        timedeltas = pd.DataFrame(result[[f"timedelta{i}" for i in range(1,i+1)]].values.flatten(order='C'))

        #
        all_df = pd.concat([ amounts, timedeltas], axis=1)
        all_df.columns = ["amounts", "timedeltas"]
        all_df.index = result.index.repeat(i)
        all_df.dropna(inplace=True)
        all_df["hours"] = (all_df["timedeltas"] /  timedelta(hours=1)).astype(int)
        all_df["months"] = all_df.index.month
        all_df["amounts_hours"] = all_df.amounts * all_df.hours

        # calc generation weighted average
        res[c] = all_df.groupby(by="months")["amounts_hours"].sum() / all_df.groupby(by="months")["amounts"].sum()

        
fig, (ax1, ax2) = plt.subplots(nrows=2,ncols=1, figsize=(10, 6))

# plot 1 (hours)
c_plot = ["battery", "Li ion"] # home battery
res[c_plot].reset_index().plot(x="index", y=c_plot, kind="bar",
                               rot=0,
                               color=[temp_colors[c] for c in c_plot],
                               xlabel="Month",
                               ylabel="Hours stored",
                               ax=ax1
                               )

# plot 2 (days)
c_plot = ["PHS", "hydro", "gas", "H2", "oil"] # "biogas",, "solid biomass"

# days
df = res[c_plot] / 24
df.reset_index().plot(x="index",
                      y=c_plot, kind="bar",
                      rot=0,
                      color=[temp_colors[c] for c in c_plot],
                      xlabel="Month",
                      ylabel="Days stored",
                      ax=ax2,
                      width=0.85
                      )

plt.show()

#fig.savefig(f"{PLOT_DIR}01_general/4.2_systems_technologies/storage_time_months_{model}.png")

In [ ]:
res

In [ ]:
res[9:12].mean()

In [ ]:
c = "H2"

start_date = "2013-02-1 00:00:00"
end_date = "2013-03-01 00:00:00"
period = n_no.generators_t.p.index[(n_no.generators_t.p.index >= start_date) & (n_no.generators_t.p.index <= end_date)]
period = "2013"

if c in n.storage_units.carrier.unique().tolist():
    # state of charge (sum over locations: 2920 values)
    charge = n.storage_units_t.state_of_charge.loc[:, n.storage_units.carrier == c].sum(axis=1)[period]

elif c in n.stores.carrier.unique().tolist():
    # state of charge (sum over locations: 2920 values)
    charge = n.stores_t.e.loc[:, n.stores.carrier == c].sum(axis=1)[period]

fig, (ax1, ax2, ax3) = plt.subplots(nrows=3,ncols=1, figsize=(12, 10))

ax1.plot(charge - charge.mean(), color="blue")
ax1.set_title(f"State of charge ({c})")
ax2.plot(charge.diff()[charge.diff() > 0], color="red", lw=2) # .resample("3h").mean()
ax2.set_title(f"Charging ({c})")
ax3.plot(charge.diff()[charge.diff() < 0].resample("3h").mean(), color="gold", lw=2)
ax3.set_title(f"Discharging ({c})")
fig.tight_layout()
plt.show()

In [ ]:
plt.plot(charge.diff())

In [ ]:
c_plot = ["biogas", "solid biomass"]

# days
df = res[c_plot] / 24
df.reset_index().plot(x="index",
                      y=c_plot, kind="bar",
                      rot=0,
                      figsize=(12,4),
                      color=[temp_colors[c] for c in c_plot],
                      xlabel="Month",
                      ylabel="Days stored",
                      )

plt.show()

In [ ]:
# furier transform to detect seasonality of state of charge

carriers = ["battery", "PHS", "hydro", "gas", "H2", "Li ion", "biogas", "solid biomass", "oil", "home battery" ]

n = n_no
period="2013"

res_charge_state = pd.DataFrame(index= charge.index)

for c in carriers:
        if c in n.storage_units.carrier.unique().tolist():
            # state of charge (sum over locations: 2920 values)
            charge = n.storage_units_t.state_of_charge.loc[:, n.storage_units.carrier == c].sum(axis=1)[period]

        elif c in n.stores.carrier.unique().tolist():
            # state of charge (sum over locations: 2920 values)
            charge = n.stores_t.e.loc[:, n.stores.carrier == c].sum(axis=1)[period]

        else:
            print(f"{c} does not exist!")

        res_charge_state[c] = charge


In [ ]:
for c, d in res_charge_state.iteritems():
    fig, ax = plt.subplots()
    fft = abs(pd.Series(np.fft.rfft(d - d.mean()), index=np.fft.rfftfreq(len(d), d=1./2920))**2)
    fft.plot(xlim=[0,1000], ylabel=f"??? Density ({c})", xlabel="Frequency (1/year)")
    print(c)
    print(pd.DataFrame(fft).sort_values(by=0, ascending=False).head(20))

#### Paper stuff: The role of storage technologies throughout the decarbonisation of the sector-coupled European Energy system
- watch out for the scenario in paper

In [ ]:
loads = ["electricity", "industry electricity", "agriculture electricity"]
n = n_no

# average hourly load (paper: 326 GWh)
avhl = n.loads_t.p.loc[:, n.loads.carrier.isin(loads)].sum().sum() / 1000 * 3 / 365 / 24
avhl

In [ ]:
# max storage capacity / energy capacity

res_stor_stst = pd.DataFrame(index = n_no.storage_units.carrier.unique().tolist() + n_no.stores.carrier.unique().tolist(), columns = ["max_stor_cap"])
res_stor_exp = pd.DataFrame(index = n_h2.storage_units.carrier.unique().tolist() + n_h2.stores.carrier.unique().tolist(), columns = ["max_stor_cap"])


for n, res_stor in zip([n_no, n_h2], [res_stor_stst, res_stor_exp]):
    
    for c in n.storage_units.carrier.unique().tolist():
        index = n.storage_units[n.storage_units.carrier == c].index
        res_stor.loc[c, "max_stor_cap"] = (n.storage_units.max_hours * n.storage_units.p_nom_opt)[index].sum() / 1000

    for c in n.stores.carrier.unique().tolist():
        res_stor.loc[c, "max_stor_cap"] = n.stores.e_nom_opt[n.stores.carrier == c].sum() / 1000

In [ ]:
# TWh (STST)
res_stor_stst.loc[["hydro", "PHS", "H2", "battery", "Li ion"]] / 1000

In [ ]:
# TWh (EXP)
res_stor_exp.loc[["hydro", "PHS", "H2", "battery", "Li ion"]] / 1000

Values from paper:

|technology   |energy capacity   |power capacity   | discharge time | ratio (energy capacity / avhl) | ratio (pow cap/avhl)
|---|---|---|---|---|---|
|'PHS'   |285 GWh   |47.5 GW   |   | ||
|'Hydro'   |   |   |   | ||
|'H2'   |6324 GWh   |156.48 GW  |   | ||
|'Battery'   |456.4 GWh   |81.5 GW   | 5.6h  | 1.4 | 0.25|
|'Li-ion'   |6100 GWh   | 1300 GW  |   | 19.4 ||

For 95% CO2 reductions, the optimal system includes electric batteries and hydrogen storage energy capacities equivalent to 1.4 and 19.4 times the average hourly electricity demand.

In [ ]:
# Power capacity (GW)

carriers = ["hydro", "PHS", "battery discharger", "V2G"]

res_pow_stst = pd.DataFrame(index = carriers, columns = ["max_stor_cap"])
res_pow_exp = pd.DataFrame(index = carriers, columns = ["max_stor_cap"])


for n, res_pow in zip([n_no, n_h2], [res_pow_stst, res_pow_exp]):
    
    for c in ["hydro", "PHS"]:
        res_pow.loc[c, "max_stor_cap"] = n.storage_units.p_nom_opt[n.storage_units.carrier == "hydro"].sum() / 1000

    for c in ["battery discharger", "V2G"]:
        res_pow.loc[c, "max_stor_cap"] = n.links[n.links.carrier == c].p_nom_opt.sum() / 1000


In [ ]:
res_pow_stst

In [ ]:
res_pow_exp

In [ ]:
# STST discharge time (h)
res_stor_stst.loc[["hydro", "PHS", "battery", "Li ion"]] # GWh
res_pow_stst.loc[["hydro", "PHS", "battery discharger", "V2G"]] # GW

res_stor_stst.loc[["hydro", "PHS", "battery", "Li ion"]].values / res_pow_stst.loc[["hydro", "PHS", "battery discharger", "V2G"]].values

In [ ]:
# EXP discharge time (h)
res_stor_exp.loc[["hydro", "PHS", "battery", "Li ion"]].values / res_pow_exp.loc[["hydro", "PHS", "battery discharger", "V2G"]].values

In [ ]:
# energy capacity ratio (normalised by avhl)
res_stor_stst.loc[["hydro", "PHS", "H2", "battery", "Li ion"]] / avhl

In [ ]:
# energy capacity ratio (normalised by avhl)
res_stor_exp.loc[["hydro", "PHS", "H2", "battery", "Li ion"]] / avhl

In [ ]:
# power capacity ratio (normalised by avhl)
res_pow_stst.loc[["hydro", "PHS", "battery discharger", "V2G"]] / avhl

In [ ]:
# power capacity ratio (normalised by avhl)
res_pow_exp.loc[["hydro", "PHS", "battery discharger", "V2G"]] / avhl

In [ ]:
# hydrogen power capacity (paper: 156.48)
# How is that calculated for a store that has no single discharger?